In [ ]:
!pip install QuantLib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 48.7 MB/s eta 0:00:00


In [ ]:
# product_definitions.py
"""
Contains classes for defining the static properties of various financial products,
including bonds and options. These classes typically set up the QuantLib
instrument if applicable, and manage the QuantLib global evaluation date
during their instantiation. They also include `from_dict` class methods
for instantiation from parameter dictionaries.
"""
import QuantLib as ql
from datetime import date
import abc

class ProductStaticBase(abc.ABC):
    """
    Abstract base class for static definitions of financial products.

    Args:
        valuation_date (date): The date for which the product is being valued.
                               This is stored and used to set QuantLib's global evaluation date
                               when product-specific QuantLib objects are typically created.
    """
    def __init__(self, valuation_date: date):
        self.valuation_date_py: date = valuation_date
        # Note: QuantLib's global evaluationDate should be set by the time
        # QL date-dependent objects are made. This is usually handled by subclasses
        # during the instantiation of QL date objects or instruments.

    @classmethod
    @abc.abstractmethod
    def from_dict(cls, params: dict) -> 'ProductStaticBase':
        """
        Abstract class method to create an instance from a dictionary of parameters.
        Subclasses must implement this method.

        Args:
            params (dict): A dictionary containing parameters to initialize the object.

        Returns:
            ProductStaticBase: An instance of a subclass of ProductStaticBase.
        """
        pass

class QuantLibBondStaticBase(ProductStaticBase):
    """
    Encapsulates common static bond definitions for QuantLib.
    This class sets up a basic fixed-rate bond schedule and instrument.

    Attributes:
        valuation_date_py (date): The valuation date (Python date object).
        maturity_date_py (date): The maturity date of the bond (Python date object).
        coupon_rate (float): The annual coupon rate (e.g., 0.03 for 3%).
        face_value (float): The face value of the bond.
        freq (int): Number of coupon payments per year.
        settlement_days (int): Number of settlement days from the valuation date.
        calendar_ql (ql.Calendar): QuantLib calendar used for date calculations.
        day_count_ql (ql.DayCounter): Day count convention used for interest calculations.
        business_convention_ql (int): QuantLib business day convention enum value.
        ql_valuation_date (ql.Date): QuantLib representation of the valuation date.
        ql_maturity_date (ql.Date): QuantLib representation of the maturity date.
        issue_date_ql (ql.Date): QuantLib representation of the issue date (defaults to valuation date).
        schedule (ql.Schedule): QuantLib payment schedule for the bond.
        bond (ql.Bond): The underlying QuantLib Bond object (typically a FixedRateBond by default).
    """
    def __init__(
        self,
        valuation_date: date,
        maturity_date: date,
        coupon_rate: float,
        face_value: float = 100.0,
        freq: int = 2,
        calendar: ql.Calendar = None, # Calendar might be derived from currency/index later
        day_count: ql.DayCounter = None,
        business_convention: int = None, # ql.BusinessDayConvention enum
        settlement_days: int = 0,
        currency: str = "USD",
        index_stub: str = None #(e.g., "SOFR", "EURIBOR6M")
    ):
        """
        Initializes the static definition for a generic QuantLib bond.

        Args:
            valuation_date (date): The valuation date.
            maturity_date (date): The maturity date of the bond.
            coupon_rate (float): The annual coupon rate (e.g., 0.03 for 3%).
            face_value (float, optional): The face value of the bond. Defaults to 100.0.
            freq (int, optional): Number of coupon payments per year. Defaults to 2 (semi-annual).
            calendar (ql.Calendar, optional): QuantLib calendar for schedule generation.
                                              Defaults to ql.TARGET().
            day_count (ql.DayCounter, optional): Day count convention for coupons and interest.
                                               Defaults to ql.ActualActual(ql.ActualActual.ISDA).
            business_convention (ql.BusinessDayConvention, optional): Business day convention for dates.
                                                                   Defaults to ql.Following.
            settlement_days (int, optional): Number of settlement days from valuation date. Defaults to 0.
        """
        super().__init__(valuation_date)
        self.maturity_date_py: date = maturity_date
        self.coupon_rate: float = coupon_rate
        self.face_value: float = face_value
        self.freq: int = freq
        self.settlement_days: int = settlement_days
        self.currency: str = currency
        self.index_stub: str = index_stub
        self.ql_valuation_date: ql.Date = ql.Date(
            self.valuation_date_py.day, self.valuation_date_py.month, self.valuation_date_py.year
        )

        # Potentially adjust default calendar based on currency if not provided
        # For example:
        if calendar is None:
            if self.currency == "USD":
                self.calendar_ql: ql.Calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
            elif self.currency == "EUR":
                self.calendar_ql: ql.Calendar = ql.Germany()
            else:
                self.calendar_ql: ql.Calendar = ql.TARGET() # Default fallback
        else:
            self.calendar_ql: ql.Calendar = calendar

        self.day_count_ql: ql.DayCounter = day_count if day_count is not None else ql.ActualActual(ql.ActualActual.ISDA)
        self.business_convention_ql: int = business_convention if business_convention is not None else ql.Following

        # Set QuantLib's global evaluation date when a bond definition is created.
        # This is important for consistency across all QL calculations.
        ql.Settings.instance().evaluationDate = self.ql_valuation_date

        self.ql_maturity_date: ql.Date = ql.Date(
            self.maturity_date_py.day, self.maturity_date_py.month, self.maturity_date_py.year
        )
        # Default issue date to valuation date. Subclasses can override if they have an explicit issue_date.
        self.issue_date_ql: ql.Date = self.ql_valuation_date

        months_in_period = int(12 / self.freq)
        # Schedule start date for vanilla/callable often aligns with valuation for pricing as of today.
        # However, for bonds with a specific issue date (like convertibles), schedule should start from issue_date_ql.
        # For FixedRateBond, the schedule usually starts from an effective date, often the issue date.
        # If settlement_days > 0, the bond starts accruing from an earlier date.
        # For simplicity here, we use issue_date_ql (which defaults to valuation_date_ql) as schedule start.
        schedule_start_date = self.issue_date_ql

        self.schedule: ql.Schedule = ql.Schedule(
            schedule_start_date, self.ql_maturity_date,
            ql.Period(months_in_period, ql.Months),
            self.calendar_ql,
            self.business_convention_ql,
            self.business_convention_ql, # terminationDateBusinessConvention
            ql.DateGeneration.Forward,
            False, # endOfMonth
        )

        # This creates a generic FixedRateBond instrument.
        # Subclasses like Callable or Convertible will override self.bond.
        self.bond: ql.Bond = ql.FixedRateBond(
            self.settlement_days,
            self.face_value,
            self.schedule,
            [self.coupon_rate], # coupons vector
            self.day_count_ql,
            self.business_convention_ql, # paymentConvention
            self.face_value # redemption value at maturity
        )

    @classmethod
    def from_dict(cls, params: dict) -> 'QuantLibBondStaticBase':
        """
        Creates a QuantLibBondStaticBase instance from a dictionary.

        Args:
            params (dict): A dictionary containing the parameters:
                'valuation_date' (date), 'maturity_date' (date), 'coupon_rate' (float).
                Optional: 'face_value' (float), 'freq' (int), 'calendar' (ql.Calendar),
                          'day_count' (ql.DayCounter), 'business_convention' (int),
                          'settlement_days' (int).

        Returns:
            QuantLibBondStaticBase: An instance of the class.
        """
        return cls(
            valuation_date=params['valuation_date'],
            maturity_date=params['maturity_date'],
            coupon_rate=params['coupon_rate'],
            face_value=params.get('face_value', 100.0),
            freq=params.get('freq', 2),
            calendar=params.get('calendar'), # User can still override currency-based default
            day_count=params.get('day_count'),
            business_convention=params.get('business_convention'),
            settlement_days=params.get('settlement_days', 0),
            currency=params.get('currency', "USD"),
            index_stub=params.get('index_stub')
        )

class CallableBondStaticBase(QuantLibBondStaticBase):
    """
    Extends QuantLibBondStaticBase for callable fixed-rate bonds.
    It adds a call schedule to the bond definition.

    Attributes:
        call_dates_py (list[date]): Python date objects for call dates.
        call_prices_py (list[float]): Call prices.
        call_schedule (ql.CallabilitySchedule): QuantLib callability schedule.
        bond (ql.CallableFixedRateBond): The QuantLib CallableFixedRateBond object.
    """
    def __init__(
        self, valuation_date: date, maturity_date: date, coupon_rate: float,
        call_dates: list[date], call_prices: list[float], face_value: float = 100.0,
        freq: int = 2, calendar: ql.Calendar = None, day_count: ql.DayCounter = None,
        business_convention: int = None, settlement_days: int = 0,
        currency: str = "USD", index_stub: str = None
    ):
        """
        Initializes the static definition for a callable bond.
        (Args documentation same as QuantLibBondStaticBase with addition of call_dates and call_prices)
        """
        super().__init__(valuation_date, maturity_date, coupon_rate, face_value, freq,
                         calendar, day_count, business_convention, settlement_days)

        self.call_dates_py: list[date] = call_dates
        self.call_prices_py: list[float] = call_prices

        self.call_schedule: ql.CallabilitySchedule = ql.CallabilitySchedule()
        for cd_py, cp in zip(call_dates, call_prices):
            ql_cd = ql.Date(cd_py.day, cd_py.month, cd_py.year)
            callability_price  = ql.BondPrice(cp, ql.BondPrice.Clean)
            call = ql.Callability(callability_price, ql.Callability.Call, ql_cd)
            self.call_schedule.push_back(call)

        # Re-create the bond as a CallableFixedRateBond
        self.bond: ql.CallableFixedRateBond = ql.CallableFixedRateBond(
            self.settlement_days,
            self.face_value,
            self.schedule, # Schedule from parent
            [self.coupon_rate],
            self.day_count_ql, # Day count from parent
            self.business_convention_ql, # Convention from parent
            self.face_value, # redemption
            self.issue_date_ql, # issue date from parent (can be set more specifically if needed)
            self.call_schedule
        )

    @classmethod
    def from_dict(cls, params: dict) -> 'CallableBondStaticBase':
        """
        Creates a CallableBondStaticBase instance from a dictionary.

        Args:
            params (dict): Dictionary containing parameters for QuantLibBondStaticBase.from_dict,
                           plus 'call_dates' (list[date]) and 'call_prices' (list[float]).

        Returns:
            CallableBondStaticBase: An instance of the class.
        """
        return cls(
            valuation_date=params['valuation_date'],
            maturity_date=params['maturity_date'],
            coupon_rate=params['coupon_rate'],
            call_dates=params['call_dates'],
            call_prices=params['call_prices'],
            face_value=params.get('face_value', 100.0),
            freq=params.get('freq', 2),
            calendar=params.get('calendar'),
            day_count=params.get('day_count'),
            business_convention=params.get('business_convention'),
            settlement_days=params.get('settlement_days', 0),
            currency=params.get('currency', "USD"),
            index_stub=params.get('index_stub')
        )

class ConvertibleBondStaticBase(QuantLibBondStaticBase):
    """
    Encapsulates static definition for a Convertible Fixed Coupon Bond.

    Attributes:
        issue_date_py (date): Python issue date.
        issue_date_ql (ql.Date): QuantLib issue date.
        conversion_ratio (float): Number of shares per bond.
        dividend_yield (float): Continuous dividend yield of underlying stock.
        equity_volatility (float): Volatility of underlying stock.
        initial_stock_price (float): Base stock price for this bond definition.
                                     Can be overridden by scenario S0 for TFF pricing.
        credit_spread_value (float): Credit spread for discounting.
        exercise_type_str (str): Type of conversion exercise (e.g., 'EuropeanAtMaturity').
        exercise (ql.Exercise): QuantLib exercise object.
        convertible_call_schedule (ql.CallabilitySchedule): Bond-level callability (not conversion features).
        bond (ql.ConvertibleFixedCouponBond): The QuantLib ConvertibleFixedCouponBond object.
    """
    def __init__(
        self, valuation_date: date, issue_date: date, maturity_date: date, coupon_rate: float,
        conversion_ratio: float, dividend_yield: float, equity_volatility: float,
        initial_stock_price: float, credit_spread_value: float, face_value: float = 100.0,
        freq: int = 2, settlement_days: int = 0, calendar: ql.Calendar = None,
        day_count: ql.DayCounter = None, business_convention: int = None,
        exercise_type: str = 'EuropeanAtMaturity',
        currency: str = "USD",
        index_stub: str = None
    ):
        """
        Initializes the static definition for a convertible bond.
        (Args documentation same as previous version)
        """
        # Call parent, but issue_date will be handled specifically for convertible
        super().__init__(valuation_date, maturity_date, coupon_rate, face_value, freq,
                         calendar, day_count, business_convention, settlement_days)

        self.issue_date_py: date = issue_date
        # Override issue_date_ql from parent as convertibles have an explicit issue date
        self.issue_date_ql: ql.Date = ql.Date(issue_date.day, issue_date.month, issue_date.year)

        # Convertible specific attributes
        self.conversion_ratio: float = conversion_ratio
        self.dividend_yield: float = dividend_yield
        self.equity_volatility: float = equity_volatility
        self.initial_stock_price: float = initial_stock_price
        self.credit_spread_value: float = credit_spread_value
        self.exercise_type_str: str = exercise_type

        # Rebuild schedule based on the convertible's actual issue_date
        months_in_period = int(12 / self.freq)
        self.schedule: ql.Schedule = ql.Schedule(
            self.issue_date_ql, # Schedule starts from the bond's issue_date
            self.ql_maturity_date,
            ql.Period(months_in_period, ql.Months),
            self.calendar_ql,
            self.business_convention_ql,
            self.business_convention_ql, # terminationDateBusinessConvention
            ql.DateGeneration.Forward,
            False # endOfMonth
        )

        # Define Exercise object
        if self.exercise_type_str == 'EuropeanAtMaturity':
            self.exercise: ql.Exercise = ql.EuropeanExercise(self.ql_maturity_date)
        # TODO: Add other exercise types here if needed (e.g., American, Bermudan)
        else:
            raise ValueError(f"Unsupported exercise type: {self.exercise_type_str}")

        # For this example, convertibles are not callable/puttable via bond features.
        # If bond-level calls/puts are needed, this schedule would be populated.
        self.convertible_call_schedule: ql.CallabilitySchedule = ql.CallabilitySchedule()

        # Create the ConvertibleFixedCouponBond instrument
        self.bond: ql.ConvertibleFixedCouponBond = ql.ConvertibleFixedCouponBond(
            self.exercise,
            self.conversion_ratio,
            self.convertible_call_schedule,
            self.issue_date_ql,
            self.settlement_days,
            [self.coupon_rate],
            self.day_count_ql,
            self.schedule,
            self.face_value
        )

    @classmethod
    def from_dict(cls, params: dict) -> 'ConvertibleBondStaticBase':
        """
        Creates a ConvertibleBondStaticBase instance from a dictionary.

        Args:
            params (dict): Dictionary containing parameters for QuantLibBondStaticBase.from_dict,
                           plus 'issue_date' (date), 'conversion_ratio' (float),
                           'dividend_yield' (float), 'equity_volatility' (float),
                           'initial_stock_price' (float), 'credit_spread_value' (float),
                           'exercise_type' (str, optional).

        Returns:
            ConvertibleBondStaticBase: An instance of the class.
        """
        return cls(
            valuation_date=params['valuation_date'],
            issue_date=params['issue_date'],
            maturity_date=params['maturity_date'],
            coupon_rate=params['coupon_rate'],
            conversion_ratio=params['conversion_ratio'],
            dividend_yield=params['dividend_yield'],
            equity_volatility=params['equity_volatility'],
            initial_stock_price=params['initial_stock_price'],
            credit_spread_value=params['credit_spread_value'],
            face_value=params.get('face_value', 100.0),
            freq=params.get('freq', 2),
            settlement_days=params.get('settlement_days', 0),
            calendar=params.get('calendar'),
            day_count=params.get('day_count'),
            business_convention=params.get('business_convention'),
            exercise_type=params.get('exercise_type', 'EuropeanAtMaturity'),
            currency=params.get('currency', "USD"),
            index_stub=params.get('index_stub')
        )

class EuropeanOptionStatic(ProductStaticBase):
    """
    Encapsulates static parameters for a European option.

    Attributes:
        expiry_date_py (date): Python expiry date.
        strike_price (float): The strike price.
        option_type (str): "call" or "put".
        day_count_convention_ql (ql.DayCounter): Day count for TTM calculation.
        time_to_expiry (float): Time to expiry in years, calculated on instantiation.
    """
    def __init__(self, valuation_date: date, expiry_date: date,
                 strike_price: float, option_type: str,
                 day_count_convention: ql.DayCounter = None,
                 currency: str = "USD"):
        """
        Initializes the static definition for a European option.

        Args:
            valuation_date (date): The valuation date.
            expiry_date (date): The expiry date of the option.
            strike_price (float): The strike price.
            option_type (str): "call" or "put".
            day_count_convention (ql.DayCounter, optional): Day count for time to expiry.
                                                          Defaults to ql.Actual365Fixed().
        """
        super().__init__(valuation_date)
        self.expiry_date_py: date = expiry_date
        self.strike_price: float = strike_price
        self.currency: str = currency

        if option_type.lower() not in ['call', 'put']:
            raise ValueError("Option type must be 'call' or 'put'")
        self.option_type: str = option_type.lower()

        # Set QL evaluation date for this calculation context
        ql_valuation_date = ql.Date(self.valuation_date_py.day, self.valuation_date_py.month, self.valuation_date_py.year)
        ql.Settings.instance().evaluationDate = ql_valuation_date # Set global eval date

        ql_expiry_date = ql.Date(self.expiry_date_py.day, self.expiry_date_py.month, self.expiry_date_py.year)

        self.day_count_convention_ql: ql.DayCounter = day_count_convention if day_count_convention is not None else ql.Actual365Fixed()

        self.time_to_expiry: float = self.day_count_convention_ql.yearFraction(ql_valuation_date, ql_expiry_date)
        # If valuation date is past expiry, time to expiry should be 0 or negative.
        # Black-Scholes typically expects T >= 0.
        if self.time_to_expiry < 0:
            self.time_to_expiry = 0.0 # Treat as expired

    @classmethod
    def from_dict(cls, params: dict) -> 'EuropeanOptionStatic':
        """
        Creates an EuropeanOptionStatic instance from a dictionary.

        Args:
            params (dict): A dictionary containing the parameters:
                'valuation_date' (date), 'expiry_date' (date),
                'strike_price' (float), 'option_type' (str: "call" or "put"),
                'day_count_convention' (ql.DayCounter, optional).

        Returns:
            EuropeanOptionStatic: An instance of the class.
        """
        return cls(
            valuation_date=params['valuation_date'],
            expiry_date=params['expiry_date'],
            strike_price=params['strike_price'],
            option_type=params['option_type'],
            day_count_convention=params.get('day_count_convention'),
            currency=params.get('currency', "USD")
        )


In [ ]:
# pricers.py
"""
Contains pricer classes for different financial products.
Pricers take a static product definition and market data to calculate a price.
"""
import QuantLib as ql
import numpy as np
from scipy.stats import norm # For Black-Scholes
import abc

class PricerBase(abc.ABC):
    """
    Abstract base class for all pricers.

    Args:
        product_static (ProductStaticBase): The static definition of the product to be priced.
    """
    def __init__(self, product_static: ProductStaticBase):
        self.product_static: ProductStaticBase = product_static

    @abc.abstractmethod
    def price(self, **kwargs) -> np.ndarray:
        """
        Abstract method to calculate the price of the product.
        Specific arguments will depend on the pricer and product type.

        Returns:
            np.ndarray: The calculated price(s). Should return a NumPy array,
                        even if it's a single-element array for a scalar price.
        """
        pass

class FastBondPricer(PricerBase):
    """
    Fast NumPy-based pricer for vanilla fixed-rate bonds using discount factors
    interpolated from a zero-coupon curve.

    Args:
        bond_static (QuantLibBondStaticBase): Static definition of the vanilla bond.
                                              While it takes QuantLibBondStaticBase, it primarily uses
                                              the generated cashflow schedule and does not rely on
                                              the QL bond object for pricing itself.
    """
    def __init__(self, bond_static: QuantLibBondStaticBase):
        if not isinstance(bond_static, QuantLibBondStaticBase):
            raise TypeError("FastBondPricer requires a QuantLibBondStaticBase derivative.")
        super().__init__(bond_static)
        self._gen_cashflows()

    def _gen_cashflows(self):
        """
        Generates and stores the bond's cashflow schedule (dates, times, amounts)
        based on the QuantLib schedule in `bond_static`.
        This is done once at initialization.
        """
        # Type hinting for clarity that product_static is a bond here
        bond_def: QuantLibBondStaticBase = self.product_static

        ql_sched = bond_def.schedule
        dc       = bond_def.day_count_ql
        ql_val   = bond_def.ql_valuation_date
        coupon_amt = bond_def.face_value * bond_def.coupon_rate / bond_def.freq

        cf_dates, cf_times, cf_amts = [], [], []
        for i in range(len(ql_sched)): # Iterate through QuantLib schedule
            d = ql_sched[i]
            # Cashflows are typically on or after the schedule's effective date.
            # We only consider cashflows strictly after the valuation date for discounting.
            if d <= ql_val:
                continue

            cf_dates.append(d.to_date())
            t = dc.yearFraction(ql_val, d) # Time from valuation to cashflow date
            cf_times.append(t)

            current_cf_amount = coupon_amt
            # Check if this is the maturity date by comparing with the bond's maturity date
            # (or the last date in the schedule if schedule is correctly aligned with maturity)
            if d == bond_def.ql_maturity_date or d == ql_sched[-1]:
                current_cf_amount += bond_def.face_value # Add principal repayment
            cf_amts.append(current_cf_amount)

        self.cf_dates: list[date] = cf_dates
        self.cf_times: np.ndarray = np.array(cf_times, dtype=float)
        self.cf_amounts: np.ndarray = np.array(cf_amts,  dtype=float)

        # Final check to ensure no cashflows at or before valuation (e.g. due to floating point precision)
        valid_cfs = self.cf_times > 1e-9 # Use a small epsilon
        self.cf_times = self.cf_times[valid_cfs]
        self.cf_amounts = self.cf_amounts[valid_cfs]
        self.cf_dates = [d for i, d_val in enumerate(self.cf_dates) if valid_cfs[i]]


    def price(self, pillar_times: np.ndarray, market_scenario_data: np.ndarray, **kwargs) -> np.ndarray:
        """
        Prices the bond by discounting its cashflows using the provided zero-coupon curve(s).

        Args:
            pillar_times (np.ndarray): 1D array of pillar times (year fractions) for the zero curve.
            market_scenario_data (np.ndarray): Zero rates.
                                              - If 1D: A single zero curve corresponding to pillar_times.
                                              - If 2D: Multiple zero curve scenarios (N_scenarios, N_pillars).
                                              Note: If TFF for other products passes S0/Vol here, this pricer will
                                              only use the rate portion based on pillar_times length.
        Returns:
            np.ndarray: Bond price(s) as a 1D NumPy array.
        """
        actual_zero_rates = market_scenario_data
        num_pillars = len(pillar_times)
        # Slice only the rate portion if more columns are present (e.g., S0 for convertible TFF context)
        if market_scenario_data.ndim == 2 and market_scenario_data.shape[1] > num_pillars:
            actual_zero_rates = market_scenario_data[:, :num_pillars]
        elif market_scenario_data.ndim == 1 and len(market_scenario_data) > num_pillars:
            actual_zero_rates = market_scenario_data[:num_pillars]

        if not self.cf_times.size: # No future cashflows
            return np.zeros(actual_zero_rates.shape[0]) if actual_zero_rates.ndim == 2 else np.array([0.0])

        if actual_zero_rates.ndim == 1:
            r_cf = np.interp(self.cf_times, pillar_times, actual_zero_rates)
            dfs  = np.exp(-r_cf * self.cf_times)
            return np.array([float(self.cf_amounts.dot(dfs))])

        r_cf_matrix = np.array([
            np.interp(self.cf_times, pillar_times, scenario_curve)
            for scenario_curve in actual_zero_rates
        ])
        dfs_matrix = np.exp(-r_cf_matrix * self.cf_times[None, :])
        prices = dfs_matrix.dot(self.cf_amounts)
        return prices

class QuantLibBondPricer(PricerBase):
    """
    Pricer using QuantLib for various bond types (vanilla, callable, convertible).
    It constructs a QuantLib yield term structure and uses the appropriate QL pricing engine.

    Args:
        bond_static (QuantLibBondStaticBase): Static definition of the bond
                                             (can be Vanilla, Callable, or Convertible).
        method (str, optional): Pricing method.
                                Defaults to 'discount'. Other options: 'g2' (for callable),
                                'convertible_binomial'.
        grid_steps (int, optional): Time steps for G2 model tree engine. Defaults to 100.
        convertible_engine_steps (int, optional): Time steps for BinomialConvertibleEngine.
                                                 Defaults to 100.
    """
    def __init__(self, bond_static: QuantLibBondStaticBase, method: str = 'discount',
                 grid_steps: int = 100, convertible_engine_steps: int = 100):
        if not isinstance(bond_static, QuantLibBondStaticBase):
            raise TypeError("QuantLibBondPricer requires a QuantLibBondStaticBase derivative.")
        super().__init__(bond_static)
        self.method: str = method.lower()
        self.grid_steps: int = grid_steps # For G2 model (callable bonds)
        self.convertible_engine_steps: int = convertible_engine_steps # For Binomial Convertible Engine
        self.is_callable_bond_type: bool = isinstance(bond_static, CallableBondStaticBase)
        self.is_convertible_bond_type: bool = isinstance(bond_static, ConvertibleBondStaticBase)

    def _make_term_structure(self, pillar_times: np.ndarray, rates_vec: np.ndarray) -> ql.YieldTermStructureHandle:
        """
        Builds a QuantLib date-based ZeroCurve from year-fraction pillar times and rates.
        `pillar_times` must be a NumPy array of floats.
        """
        pillar_times_np = np.asarray(pillar_times, dtype=float)

        # product_static is QuantLibBondStaticBase, so ql_valuation_date exists
        base_date: ql.Date = self.product_static.ql_valuation_date
        dates = ql.DateVector()
        effective_rates = list(rates_vec) # Make a mutable copy

        # Add (base_date, first_rate) if pillars don't start at t=0
        if not pillar_times_np.size or (pillar_times_np.size > 0 and pillar_times_np[0] > 1e-6):
             dates.push_back(base_date)
             effective_rates.insert(0, rates_vec[0] if rates_vec.size > 0 else 0.0)

        for t_val in pillar_times_np: # Iterate over the NumPy array
            dates.push_back(base_date + ql.Period(int(round(t_val*365.0)), ql.Days))

        # Ensure rates and dates vectors match, QL requires this.
        if len(effective_rates) != len(dates):
             # This might occur if pillar_times was empty and rates_vec was also empty.
             # The logic above tries to ensure effective_rates has one element for the base_date.
             if len(dates)==1 and not effective_rates: # Only base_date, no rates provided
                 effective_rates.append(0.0) # Default rate for base_date
             # Other mismatches could indicate issues with input data.
             # For robustness, one might add more sophisticated alignment logic or raise an error.

        # Use day_count and calendar from the bond's static definition
        zc = ql.ZeroCurve(dates, effective_rates, self.product_static.day_count_ql,
                          self.product_static.calendar_ql, ql.Linear(), ql.Continuous, ql.Annual)
        zc.enableExtrapolation()
        return ql.YieldTermStructureHandle(zc)

    @staticmethod
    def _price_vanilla_static(bond_instrument: ql.Bond, ts_handle: ql.YieldTermStructureHandle) -> float:
        """Static method to price a vanilla bond using DiscountingBondEngine."""
        engine = ql.DiscountingBondEngine(ts_handle)
        bond_instrument.setPricingEngine(engine)
        return bond_instrument.NPV()

    @staticmethod
    def _price_callable_static(bond_instrument: ql.CallableFixedRateBond, ts_handle: ql.YieldTermStructureHandle,
                               model_params: tuple, grid_steps: int) -> float:
        """Static method to price a G2++ callable bond."""
        a,sigma,b,eta,rho = model_params
        model=ql.G2(ts_handle,a,sigma,b,eta,rho)
        engine=ql.TreeCallableFixedRateBondEngine(model,grid_steps)
        bond_instrument.setPricingEngine(engine)
        return bond_instrument.cleanPrice()

    @staticmethod
    def _price_convertible_static(
        bond_instrument: ql.ConvertibleFixedCouponBond,
        ts_handle: ql.YieldTermStructureHandle, # Risk-free rate for equity process
        static_def: ConvertibleBondStaticBase, # Contains equity/credit params
        eng_steps: int,
        s0_scen: float = None # Scenario-specific S0, if provided
        ) -> float:
        """Static method to price a convertible bond using BinomialCRRConvertibleEngine."""
        eval_d: ql.Date = ql.Settings.instance().evaluationDate # Get current global eval date

        # Use scenario S0 if provided, otherwise use the S0 from the static definition
        current_s0 = s0_scen if s0_scen is not None else static_def.initial_stock_price
        s0_h = ql.QuoteHandle(ql.SimpleQuote(current_s0))

        # Dividend yield term structure
        div_h = ql.YieldTermStructureHandle(
            ql.FlatForward(eval_d, static_def.dividend_yield, static_def.day_count_ql)
        )
        # Volatility term structure
        vol_h = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(eval_d, static_def.calendar_ql, static_def.equity_volatility, static_def.day_count_ql)
        )

        # Equity process (Black-Scholes-Merton)
        # ts_handle here is the risk-free rate for the equity process
        proc = ql.BlackScholesMertonProcess(s0_h, div_h, ts_handle, vol_h)

        # Credit Spread
        cs_h = ql.QuoteHandle(ql.SimpleQuote(static_def.credit_spread_value))

        # Pricing Engine
        engine = ql.BinomialCRRConvertibleEngine(proc, eng_steps, cs_h)
        bond_instrument.setPricingEngine(engine)
        return bond_instrument.cleanPrice()

    def price(self, pillar_times: np.ndarray, market_scenario_data: np.ndarray, g2_params=None) -> np.ndarray:
        """
        Prices the bond based on the configured method and market scenario data.

        Args:
            pillar_times (np.ndarray): 1D array of pillar times for the interest rate curve.
            market_scenario_data (np.ndarray): Market data for scenarios.
                - If 1D: Contains rates, or rates + S0 (if convertible and S0 is a TFF factor).
                - If 2D: Each row contains rates, or rates + S0.
            g2_params (tuple or list, optional): G2 model parameters, only for 'g2' method.

        Returns:
            np.ndarray: Calculated price(s) as a 1D NumPy array.
        """
        pillar_times_np = np.asarray(pillar_times, dtype=float)
        num_rate_pillars = len(pillar_times_np)

        if market_scenario_data.ndim == 1:
            # Handle single scenario, potentially with S0 if convertible
            rates_scen = market_scenario_data[:num_rate_pillars]
            s0_scen = market_scenario_data[num_rate_pillars] if len(market_scenario_data) == num_rate_pillars + 1 else None
            if len(rates_scen) != num_rate_pillars:
                raise ValueError("Single scenario rates length mismatch with pillar_times.")
            return np.array([float(self._price_single_curve_logic(pillar_times_np, rates_scen, g2_params, s0_scen))])

        # Multi-scenario
        prices = []
        # Determine if g2_params is a single set for all scenarios or a list per scenario
        single_g2_params_set = isinstance(g2_params, tuple) or \
                             (isinstance(g2_params, list) and g2_params and isinstance(g2_params[0], (float,int)))
        if self.method == 'g2' and g2_params is not None and not single_g2_params_set and len(g2_params) != market_scenario_data.shape[0]:
            raise ValueError("List of g2_params must match number of scenarios.")

        for i, scen_data_row in enumerate(market_scenario_data):
            rates_scen = scen_data_row[:num_rate_pillars]
            s0_scen = scen_data_row[num_rate_pillars] if scen_data_row.shape[0] == num_rate_pillars + 1 else None
            if rates_scen.shape[0] != num_rate_pillars:
                raise ValueError(f"Scenario {i} rates length mismatch with pillar_times.")

            current_g2_p = g2_params if single_g2_params_set else (g2_params[i] if g2_params and self.method == 'g2' else None)
            prices.append(self._price_single_curve_logic(pillar_times_np, rates_scen, current_g2_p, s0_scen))
        return np.array(prices)

    def _price_single_curve_logic(self, pillar_times_np_arg: np.ndarray, zero_rates_scen: np.ndarray,
                                  g2_p=None, s0_scen:float=None) -> float:
        """Helper method to price for a single scenario's rates and optional S0."""
        # Ensure QL Eval Date is set for current process/thread context
        # product_static.ql_valuation_date should be a ql.Date object
        ql.Settings.instance().evaluationDate = self.product_static.ql_valuation_date

        ts_handle = self._make_term_structure(pillar_times_np_arg, zero_rates_scen)

        if self.is_convertible_bond_type and self.method == 'convertible_binomial':
            if not isinstance(self.product_static, ConvertibleBondStaticBase): # Type check for safety
                raise TypeError("Pricer's product_static is not a ConvertibleBondStaticBase instance for convertible pricing.")
            return self._price_convertible_static(
                self.product_static.bond, # This is ql.ConvertibleFixedCouponBond
                ts_handle,
                self.product_static, # This is ConvertibleBondStaticBase instance
                self.convertible_engine_steps,
                s0_scen # Pass scenario-specific S0
            )
        elif self.is_callable_bond_type and self.method == 'g2':
            if g2_p is None: raise ValueError("g2_params needed for G2 callable pricing.")
            if not isinstance(self.product_static.bond, ql.CallableFixedRateBond):
                raise TypeError("Pricer's product_static.bond is not a ql.CallableFixedRateBond.")
            return self._price_callable_static(self.product_static.bond, ts_handle, g2_p, self.grid_steps)
        elif self.method == 'discount': # Assuming vanilla bond or pricing non-optional part
            if not isinstance(self.product_static.bond, ql.Bond): # General check
                 raise TypeError("Pricer's product_static.bond is not a ql.Bond instance for discounting.")
            return self._price_vanilla_static(self.product_static.bond, ts_handle)

        raise ValueError(f"Unsupported pricing method '{self.method}' or product type for QuantLibBondPricer.")

class BlackScholesPricer(PricerBase):
    """
    Pricer for European options using the Black-Scholes formula.

    Args:
        option_static (EuropeanOptionStatic): Static parameters of the European option.
        risk_free_rate (float): The risk-free interest rate (annualized, continuous compounding).
        dividend_yield (float, optional): Continuous dividend yield of the underlying. Defaults to 0.0.
    """
    def __init__(self, option_static: EuropeanOptionStatic,
                 risk_free_rate: float, dividend_yield: float = 0.0):
        if not isinstance(option_static, EuropeanOptionStatic):
            raise TypeError("BlackScholesPricer requires an EuropeanOptionStatic instance.")
        super().__init__(option_static)
        self.risk_free_rate: float = risk_free_rate
        self.dividend_yield: float = dividend_yield
    def price(self, stock_price: np.ndarray, volatility: np.ndarray, **kwargs) -> np.ndarray:
        S_arr = np.asarray(stock_price)
        sigma_input = np.asarray(volatility)

        option_def: EuropeanOptionStatic = self.product_static
        K = option_def.strike_price
        T = option_def.time_to_expiry
        r = self.risk_free_rate
        q = self.dividend_yield
        opt_type = option_def.option_type

        is_scalar_input = (S_arr.ndim == 0 and sigma_input.ndim == 0)

        if S_arr.ndim == 0 and sigma_input.ndim > 0: S_arr = np.full_like(sigma_input, S_arr)
        if sigma_input.ndim == 0 and S_arr.ndim > 0: sigma_input = np.full_like(S_arr, sigma_input)

        price_val = np.zeros_like(S_arr, dtype=float) if not is_scalar_input else 0.0

        if T <= 1e-9:
            if opt_type == 'call': price_val = np.maximum(S_arr - K, 0.0)
            else: price_val = np.maximum(K - S_arr, 0.0)
            # If the problematic line was here:
            # return np.array([price_val]) if is_scalar_input and isinstance(price_val, (float, np.float64)) else price_val
            return price_val


        is_vol_effectively_zero = (sigma_input <= 1e-9)
        sigma_calc = np.maximum(sigma_input, 1e-16)

        with np.errstate(divide='ignore', invalid='ignore'):
            d1 = (np.log(S_arr / K) + (r - q + 0.5 * sigma_calc**2) * T) / (sigma_calc * np.sqrt(T))
            d2 = d1 - sigma_calc * np.sqrt(T)
            d1 = np.where(S_arr <= 0, -np.inf if opt_type == 'call' else np.inf, d1)
            d2 = np.where(S_arr <= 0, -np.inf if opt_type == 'call' else np.inf, d2)

        if opt_type == 'call':
            price_bs = S_arr * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        else: # put
            price_bs = K * np.exp(-r * T) * norm.cdf(-d2) - S_arr * np.exp(-q * T) * norm.cdf(-d1)

        if np.any(is_vol_effectively_zero):
            intrinsic_val = None
            if opt_type == 'call':
                intrinsic_val = np.maximum(0.0, S_arr * np.exp(-q * T) - K * np.exp(-r * T))
            else: # put
                intrinsic_val = np.maximum(0.0, K * np.exp(-r * T) - S_arr * np.exp(-q * T))

            if is_scalar_input and is_vol_effectively_zero:
                price_val = intrinsic_val
            elif not is_scalar_input :
                price_val = np.where(is_vol_effectively_zero, intrinsic_val, price_bs)
            elif not isinstance(price_val, np.ndarray) and isinstance(S_arr, np.ndarray):
                 price_val = np.where(is_vol_effectively_zero, intrinsic_val, price_bs)
        else:
            price_val = price_bs

        # If the problematic line from the traceback was at the very end:
        # return np.array([price_val]) if is_scalar_input and isinstance(price_val, (float, np.float64)) else price_val
        # However, the existing return statement is just:
        return price_val


In [ ]:
# g2_model.py
"""
Contains the G2Calibrator class for calibrating the G2++ interest rate model.
"""
import QuantLib as ql

class G2Calibrator:
    """
    Calibrates a G2++ (two-factor Hull-White) interest rate model
    to market cap/floor quotes.

    The G2++ model has 5 parameters: a, sigma, b, eta, rho.
    - a: mean reversion speed of the first factor (x)
    - sigma: volatility of the first factor (x)
    - b: mean reversion speed of the second factor (y)
    - eta: volatility of the second factor (y)
    - rho: correlation between the Wiener processes driving x and y

    Args:
        ts_handle (ql.YieldTermStructureHandle): Handle to the initial yield term structure
                                                 to which the model will be fitted. This term
                                                 structure defines the initial discount curve.
        index (ql.IborIndex or ql.OvernightIndex): The interest rate index underlying the
                                                   calibration instruments (e.g., caps/floors).
                                                   This index is used by the CapHelper to
                                                   determine caplet cashflows and fixings.
    """
    def __init__(self, ts_handle: ql.YieldTermStructureHandle, index: ql.InterestRateIndex):
        self.ts_handle: ql.YieldTermStructureHandle = ts_handle
        self.index: ql.InterestRateIndex = index
        # Initialize a G2 model. The parameters (a, sigma, b, eta, rho) will be calibrated.
        # The model is initialized with the provided term structure.
        self.model: ql.G2 = ql.G2(self.ts_handle)

    def calibrate(
        self,
        periods: list[ql.Period],
        quotes: list[ql.QuoteHandle],
        optimization_method: ql.OptimizationMethod = None,
        end_criteria: ql.EndCriteria = None,
        engine_steps: int = 50
    ) -> tuple[float, float, float, float, float]:
        """
        Calibrates the G2 model parameters to market cap volatilities.

        Args:
            periods (list[ql.Period]): List of ql.Period objects for cap tenors
                                       (e.g., [ql.Period('1Y'), ql.Period('2Y'), ...]).
                                       There should be at least 5 instruments for 5 parameters.
            quotes (list[ql.QuoteHandle]): List of ql.QuoteHandle objects containing the market
                                           volatilities (or prices) for the corresponding periods.
                                           Assumes Normal volatility for caps.
            optimization_method (ql.OptimizationMethod, optional):
                                QuantLib optimization method (e.g., ql.LevenbergMarquardt).
                                Defaults to a standard LevenbergMarquardt.
            end_criteria (ql.EndCriteria, optional):
                                QuantLib end criteria for the optimization.
                                Defaults to a standard EndCriteria.
            engine_steps (int, optional): Number of time steps for the TreeCapFloorEngine
                                          used to price caps during calibration. Affects accuracy
                                          and speed of calibration. Defaults to 50.

        Returns:
            tuple[float, float, float, float, float]:
                A tuple containing the calibrated G2 model parameters: (a, sigma, b, eta, rho).

        Raises:
            ValueError: If the number of periods and quotes do not match.
            RuntimeError: If calibration fails in QuantLib (e.g., due to insufficient instruments
                          or optimization issues).
        """
        if len(periods) != len(quotes):
            raise ValueError("Length of periods and quotes must match for calibration.")
        if len(periods) < 5: # G2++ has 5 parameters
            # This is a common cause of calibration failure.
            print(f"Warning: Number of calibration instruments ({len(periods)}) is less than 5. "
                  "G2++ calibration might be unstable or fail due to under-specification.")

        helpers = []
        for period_obj, quote_handle in zip(periods, quotes):
            # Create a CapHelper for each market quote.
            # The CapHelper links the market quote (volatility) to the model price of the cap.
            # Parameters like cap frequency, day count should ideally match the market convention
            # of the provided quotes.
            helper = ql.CapHelper(
                period_obj,
                quote_handle,
                self.index,                     # IborIndex underlying the cap
                ql.Semiannual,                 # Cap/floor coupon frequency (e.g., Semiannual)
                self.index.dayCounter() if self.index.dayCounter() else ql.Actual360(), # Day count for caplets
                False,                         # Not used for caps (related to first fixing)
                self.ts_handle,                # Initial term structure for discounting
                ql.BlackCalibrationHelper.RelativePriceError, # Error type for calibration objective function
                ql.Normal,                     # Volatility type (e.g., Normal, ShiftedLognormal)
                0.0                            # Shift (if using ShiftedLognormal)
            )
            # The pricing engine for the helper uses the G2 model instance that is being calibrated.
            # engine_steps for the tree engine affects accuracy/speed of pricing each cap during calibration.
            engine = ql.TreeCapFloorEngine(self.model, engine_steps)
            helper.setPricingEngine(engine)
            helpers.append(helper)

        # Default optimization method and end criteria if not provided
        opt_method = optimization_method or ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
        crit = end_criteria or ql.EndCriteria(
            maxIterations=10000,
            maxStationaryStateIterations=1000,
            rootEpsilon=1e-8,
            functionEpsilon=1e-8,
            gradientNormEpsilon=1e-8
        )

        # Perform the calibration. This modifies self.model in-place.
        self.model.calibrate(helpers, opt_method, crit)

        # Return the calibrated parameters from the model
        # params() returns [a, sigma, b, eta, rho]
        return self.model.params()


In [ ]:
# tff_approximator.py
"""
Contains classes and functions for Tensor Functional Form (TFF) approximation.

This module provides:
1.  `TensorFunctionalForm`: A class representing the quadratic TFF model (x'Ax + b'x + c).
2.  `engineer_option_features`: A function to create polynomial features from raw option inputs (S0, Vol).
3.  `normalize_features`: A function to apply Z-score normalization to features.
4.  `_price_one_scenario_for_tff`: A top-level worker function designed for parallel execution
    to price a single scenario by reconstructing product and pricer objects.
5.  `TensorFunctionalFormCalibrate`: A class to calibrate (fit) the TFF model by:
    - Generating training scenarios using various sampling methods (Sobol, LHS, Uniform).
    - Pricing these scenarios using a provided "true" pricer (potentially in parallel).
    - Performing feature engineering and normalization (especially for options).
    - Solving a least-squares problem to find the TFF coefficients.
    - Evaluating the TFF's accuracy (RMSE) on a test set.
"""
import numpy as np
from scipy.stats.qmc import LatinHypercube, Sobol, scale # For advanced sampling
from concurrent.futures import ProcessPoolExecutor # For parallel processing
import QuantLib as ql # For ql.Date and ql.Settings in worker
from datetime import date # For type hinting
import re

# --- Feature Engineering and Normalization for Options ---
def engineer_option_features(
    s0_values: np.ndarray,
    vol_values: np.ndarray,
    order: int = 2
) -> tuple[np.ndarray, list[str]]:
    """
    Engineers polynomial features from stock price (S0) and volatility (Vol) for option TFF.
    This helps the quadratic TFF capture more complex option price behavior.

    Args:
        s0_values (np.ndarray): 1D array of stock prices.
        vol_values (np.ndarray): 1D array of volatilities, same shape as s0_values.
        order (int, optional): The maximum order of individual S0 or Vol terms to include
                               and guides the generation of cross-product terms.
                               For example:
                               - order=1: [S0, Vol]
                               - order=2: [S0, Vol, S0^2, Vol^2, S0*Vol]
                               - order=3: Adds S0^3, Vol^3, S0^2*Vol, S0*Vol^2
                               - order=4: Adds S0^4, Vol^4, S0^3*Vol, S0*Vol^3, S0^2*Vol^2
                               Defaults to 2.

    Returns:
        tuple[np.ndarray, list[str]]:
            - A 2D NumPy array where each row contains the engineered features for an input (S0, Vol) pair.
            - A list of strings describing the engineered features (names), useful for debugging or analysis.
    """
    s0 = np.asarray(s0_values)
    vol = np.asarray(vol_values)

    if s0.shape != vol.shape:
        raise ValueError("s0_values and vol_values must have the same shape for feature engineering.")
    if s0.ndim != 1: # Expecting 1D arrays of scenarios for S0 and Vol
        raise ValueError("s0_values and vol_values must be 1D arrays.")

    # Base features (order 1)
    features = [s0, vol]
    feature_names = ['S0', 'Vol']

    # Higher order features
    if order >= 2:
        features.extend([s0**2, vol**2, s0 * vol])
        feature_names.extend(['S0^2', 'Vol^2', 'S0*Vol'])
    if order >= 3:
        features.extend([s0**3, vol**3, (s0**2) * vol, s0 * (vol**2)])
        feature_names.extend(['S0^3', 'Vol^3', 'S0^2*Vol', 'S0*Vol^2'])
    if order >= 4:
        features.extend([s0**4, vol**4, (s0**3) * vol, s0 * (vol**3), (s0**2)*(vol**2)])
        feature_names.extend(['S0^4', 'Vol^4', 'S0^3*Vol', 'S0*Vol^3', 'S0^2*Vol^2'])
    # Add more orders if needed

    # Stack features column-wise: each row is a sample, each column is a feature
    return np.vstack(features).T, feature_names

def normalize_features(
    features: np.ndarray,
    means: np.ndarray = None,
    stds: np.ndarray = None
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Normalizes features to have zero mean and unit standard deviation (Z-score normalization).
    If means and stds are not provided, they are calculated from the input features (for training set).
    If means and stds are provided, they are used to normalize the input features (for test/prediction set).

    Args:
        features (np.ndarray): 2D array of features, where rows are samples and columns are features.
        means (np.ndarray, optional): Pre-calculated means for each feature column. Defaults to None.
        stds (np.ndarray, optional): Pre-calculated standard deviations for each feature column. Defaults to None.

    Returns:
        tuple[np.ndarray, np.ndarray, np.ndarray]:
            - normalized_features: The Z-score normalized features.
            - means: The means used for normalization (calculated if not provided).
            - stds: The standard deviations used for normalization (calculated if not provided).
    """
    if features.ndim != 2:
        raise ValueError("Features input must be a 2D array (N_samples, N_features).")

    if means is None or stds is None: # Calculate for training set
        means = np.mean(features, axis=0)
        stds = np.std(features, axis=0)
        # Avoid division by zero for constant features (where std might be 0 or very small)
        stds[stds < 1e-8] = 1.0

    normalized_features = (features - means) / stds
    return normalized_features, means, stds

def _parse_numeric_pillars_from_factor_names(factor_names: list[str]) -> np.ndarray:
    """
    Parses numeric pillar times from a list of TFF raw factor names.
    It extracts numbers from strings like 'rate_0.25Y' or handles direct numeric strings like '0.25'.
    Non-rate factors like 'S0' or 'Volatility' are ignored.
    """
    parsed_pillars = []
    for name_str in factor_names:
        name_upper = name_str.upper()
        # Skip known non-rate factors that don't represent time pillars
        if 'S0' == name_upper or 'VOLATILITY' == name_upper: # Add other non-pillar factor names if any
            continue

        # Try to extract a number (float) from the string
        # This regex looks for the first sequence of digits, possibly with a decimal point
        match = re.search(r'(\d+(\.\d+)?)', name_str)
        if match:
            try:
                parsed_pillars.append(float(match.group(1)))
            except ValueError:
                # This should ideally not happen if regex matches a number
                raise ValueError(f"Could not convert extracted number '{match.group(1)}' to float from factor name: {name_str}")
        else:
            # If no number pattern is found by regex, attempt direct conversion
            # This handles cases where factor_names might be purely numeric strings like "0.5", "10.0"
            try:
                parsed_pillars.append(float(name_str))
            except ValueError:
                # If direct conversion also fails, and it's not a known non-pillar factor, then it's an unparseable pillar name
                raise ValueError(f"Could not parse pillar time from factor name: '{name_str}'. It's not a recognized non-pillar factor and could not be parsed as a number.")

    # Return sorted unique pillar times
    if not parsed_pillars:
        return np.array([], dtype=float)
    return np.array(sorted(list(set(parsed_pillars))), dtype=float)

# --- Tensor Functional Form Model ---
class TensorFunctionalForm:
    """
    Represents a multivariate quadratic function f(x) = x^T A x + b^T x + c.
    The input x is expected to be the vector of (potentially engineered and normalized) features.

    Attributes:
        A (np.ndarray): The symmetric (D,D) matrix of quadratic coefficients.
        b (np.ndarray): The (D,) vector of linear coefficients.
        c (float): The constant term.
        D (int): The dimension of the input feature vector x.
    """
    def __init__(self, A: np.ndarray, b: np.ndarray, c: float):
        """
        Initializes the TensorFunctionalForm model.

        Args:
            A (np.ndarray): A (D,D) symmetric matrix of quadratic coefficients.
            b (np.ndarray): A vector of length D representing linear coefficients.
            c (float): The constant term.
        """
        self.A: np.ndarray = A
        self.b: np.ndarray = b
        self.c: float = c
        if A.ndim != 2 or A.shape[0] != A.shape[1]:
            raise ValueError("Matrix A must be square.")
        if b.ndim != 1 or b.shape[0] != A.shape[0]:
            raise ValueError("Vector b dimension must match matrix A's dimension.")
        self.D: int = A.shape[0] # Dimension of the input feature vector x

    def __call__(self, x: np.ndarray) -> np.ndarray:
        """
        Evaluates the quadratic form: f(x) = x^T A x + b^T x + c.

        Args:
            x (np.ndarray): Input array of features.
                            - If 1D: shape (D,) - a single scenario's feature vector.
                            - If 2D: shape (N, D) - N scenarios' feature vectors.
                                     Or shape (D, N) - will be transposed if D matches self.D.

        Returns:
            np.ndarray or float: Scalar price if x is 1D, or a 1D array of prices (N,) if x is 2D.
        """
        x_arr = np.asarray(x)

        if x_arr.ndim == 1:
            if x_arr.shape[0] != self.D:
                raise ValueError(f"Input vector dimension {x_arr.shape[0]} does not match model dimension {self.D}.")
            # x.T @ A @ x
            term_quadratic = x_arr @ self.A @ x_arr
            # b.T @ x
            term_linear = self.b @ x_arr
            return float(term_quadratic + term_linear + self.c)

        elif x_arr.ndim == 2:
            if x_arr.shape[1] == self.D: # Input is (N_samples, D_features)
                # Efficient calculation for multiple samples:
                # (x @ A) results in (N,D). Then element-wise multiply by x and sum over D.
                term_quadratic = np.sum((x_arr @ self.A) * x_arr, axis=1)
                term_linear = x_arr @ self.b # Results in (N,)
                return term_quadratic + term_linear + self.c
            elif x_arr.shape[0] == self.D: # Input is (D_features, N_samples), transpose
                x_arr_T = x_arr.T
                term_quadratic = np.sum((x_arr_T @ self.A) * x_arr_T, axis=1)
                term_linear = x_arr_T @ self.b
                return term_quadratic + term_linear + self.c
            else:
                raise ValueError(f"Invalid input shape {x_arr.shape} for TFF. Expected (N_samples, {self.D}) or ({self.D}, N_samples).")
        else:
            raise ValueError(f"Input for TFF must be 1D or 2D array, got ndim={x_arr.ndim}")

    def to_dict(self) -> dict:
        """
        Serializes the TFF model to a dictionary.
        NumPy arrays are converted to lists for easier persistence.

        Returns:
            dict: A dictionary representation of the TFF model.
        """
        return {
            'A': self.A.tolist(),  # Convert NumPy array A to list of lists
            'b': self.b.tolist(),  # Convert NumPy array b to list
            'c': self.c,
            'D': self.D
        }

    @classmethod
    def from_dict(cls, data: dict) -> 'TensorFunctionalForm':
        """
        Deserializes a TFF model from a dictionary representation.

        Args:
            data (dict): A dictionary containing 'A', 'b', 'c', and 'D'.
                         'A' and 'b' should be lists (or lists of lists).

        Returns:
            TensorFunctionalForm: An instance of the TFF model.
        """
        if not all(key in data for key in ['A', 'b', 'c', 'D']):
            raise ValueError("Data dictionary is missing one or more required keys: 'A', 'b', 'c', 'D'.")

        A_list = data['A']
        b_list = data['b']
        c_scalar = data['c']
        # D_val = data['D'] # D is implicitly defined by shape of A

        try:
            A = np.array(A_list, dtype=float)
            b = np.array(b_list, dtype=float)
        except Exception as e:
            raise ValueError(f"Could not convert A or b from lists to NumPy arrays: {e}")

        # Basic validation based on D (optional, as __init__ also validates)
        # if A.shape[0] != D_val or A.shape[1] != D_val:
        #     raise ValueError(f"Shape of A {A.shape} from dictionary does not match D value {D_val}")
        # if b.shape[0] != D_val:
        #     raise ValueError(f"Shape of b {b.shape} from dictionary does not match D value {D_val}")

        return cls(A, b, c_scalar)



# --- Worker function for parallel TFF training data generation ---
# This function needs to be defined at the top level of a module for ProcessPoolExecutor pickling.
def _price_one_scenario_for_tff(worker_args: tuple) -> float:
    """
    Helper function to price a single scenario for TFF training.
    It reconstructs necessary QuantLib objects and the pricer within the worker process
    using `from_dict` class methods for product static data.
    """
    (product_static_params_dict, pricer_config_params,
     factor_names_for_tff, single_market_scenario_data, # factor_names_for_tff are the full raw TFF input names
     valuation_date_for_worker, price_kwargs_dict) = worker_args

    ql_valuation_date_worker = ql.Date(
        valuation_date_for_worker.day,
        valuation_date_for_worker.month,
        valuation_date_for_worker.year
    )
    ql.Settings.instance().evaluationDate = ql_valuation_date_worker

    product_type = product_static_params_dict['product_type']

    # actual_rate_pillars are the numeric tenors for bond pricing, passed in product_static_params_dict
    # This was populated by _parse_numeric_pillars_from_factor_names in TFFCalibrate.__init__
    actual_rate_pillars_for_worker = np.asarray(product_static_params_dict.get('actual_rate_pillars', []), dtype=float)

    product_static_obj = None
    current_static_params_for_reconstruction = product_static_params_dict.copy()
    current_static_params_for_reconstruction['valuation_date'] = valuation_date_for_worker

    # For convertible bonds, if S0 is part of its TFF factors,
    # its value from single_market_scenario_data needs to override initial_stock_price
    # We need to identify if S0 is a factor for this convertible's TFF.
    # factor_names_for_tff holds the names of factors in single_market_scenario_data.
    if product_type == 'convertible':
        s0_is_tff_factor_for_conv = False
        s0_value_from_scenario = None
        # Assuming S0 factor for convertible, if present, is the last one in factor_names_for_tff
        # This relies on how conv_tff_input_factor_names_demo_currency was constructed in main_demonstration.py
        # A more robust way would be to check names if order wasn't guaranteed or to pass a flag.
        if len(factor_names_for_tff) > len(actual_rate_pillars_for_worker): # Implies an extra factor like S0
             # Check if the last factor name suggests it's S0
             # This check needs to be robust based on your naming convention.
             # Example: if last name in factor_names_for_tff ends with "_S0"
             if factor_names_for_tff[-1].upper().endswith("_S0"): # Simple check
                s0_is_tff_factor_for_conv = True
                s0_value_from_scenario = single_market_scenario_data[-1]

        if s0_is_tff_factor_for_conv and s0_value_from_scenario is not None:
            current_static_params_for_reconstruction['initial_stock_price'] = s0_value_from_scenario
        # Else, initial_stock_price from the original product_static_params_dict (base value) will be used

    # Reconstruct product_static using from_dict
    if product_type == 'vanilla':
        product_static_obj = QuantLibBondStaticBase.from_dict(current_static_params_for_reconstruction)
    elif product_type == 'callable':
        product_static_obj = CallableBondStaticBase.from_dict(current_static_params_for_reconstruction)
    elif product_type == 'convertible':
        product_static_obj = ConvertibleBondStaticBase.from_dict(current_static_params_for_reconstruction)
    elif product_type == 'european_option':
        product_static_obj = EuropeanOptionStatic.from_dict(current_static_params_for_reconstruction)
    else:
        raise ValueError(f"Unknown product type for TFF worker: {product_type}")

    # Reconstruct Pricer object and price
    if product_type in ['vanilla', 'callable', 'convertible']:
        pricer_instance = QuantLibBondPricer(product_static_obj,
                                             **pricer_config_params.get('bond_pricer_config',{}))
        # single_market_scenario_data contains the raw TFF input values for this scenario
        # (e.g., [rate1_val, rate2_val, ..., s0_val_if_applicable_for_conv_tff])
        # The QuantLibBondPricer.price method is expected to handle this structure.
        market_data_for_ql_pricer = np.array([single_market_scenario_data])

        price_result_array = pricer_instance.price(
            pillar_times=actual_rate_pillars_for_worker, # Numeric tenors
            market_scenario_data=market_data_for_ql_pricer,
            **price_kwargs_dict
        )
        return price_result_array[0]

    elif product_type == 'european_option':
        bs_config = pricer_config_params.get('bs_pricer_config', {})
        pricer_instance = BlackScholesPricer(product_static_obj,
                                             bs_config['risk_free_rate'],
                                             bs_config.get('dividend_yield',0.0))

        s0_scenario_val = None
        vol_scenario_val = None

        # Assumption: For options, factor_names_for_tff = [s0_full_name, vol_full_name]
        # and single_market_scenario_data = [s0_value, vol_value]
        if len(factor_names_for_tff) == 2 and len(single_market_scenario_data) == 2:
            s0_scenario_val = single_market_scenario_data[0]
            vol_scenario_val = single_market_scenario_data[1]
        else:
            # This will lead to the error below if they remain None
            pass

        if s0_scenario_val is None or vol_scenario_val is None:
            raise ValueError(
                f"S0 and/or Volatility values not resolved for European Option pricing scenario.\n"
                f"  Expected 2 TFF input factors (S0, Volatility) in order. "
                f"Received {len(factor_names_for_tff)} factor names: {factor_names_for_tff} "
                f"with {len(single_market_scenario_data)} data points."
            )
        return pricer_instance.price(stock_price=s0_scenario_val, volatility=vol_scenario_val)

    raise ValueError(f"Pricer reconstruction or pricing failed for product type: {product_type}")



class TensorFunctionalFormCalibrate:
    """
    Builds a TensorFunctionalForm approximation for a given product and pricer.
    Handles feature engineering and normalization for options.

    The `pricer_template` is used to extract static parameters for reconstruction in workers.
    `tff_input_raw_factor_names` and `tff_input_raw_base_values` define the raw input space for the TFF.
    """
    def __init__(
        self,
        pricer_template: PricerBase,
        tff_input_raw_factor_names: list[str], # Names of RAW factors (e.g., rate tenors, 'S0', 'Volatility')
        tff_input_raw_base_values: np.ndarray, # Base values for these RAW factors
        base_s0_for_convertible_tff: float = None # Specific base S0 if fitting a TFF for a
                                                 # convertible bond where S0 is a dynamic factor.
    ):
        """
        Initializes the TFF calibrator.

        Args:
            pricer_template (PricerBase): An instance of the pricer for the product type
                                          (e.g., QuantLibBondPricer, BlackScholesPricer).
                                          Used to extract static product and pricer parameters.
            tff_input_raw_factor_names (list[str]): Names of the raw input factors for the TFF.
                                                    For bonds: list of rate pillar times (as floats/ints).
                                                    For bonds with S0 factor: list of rate pillars + 'S0'.
                                                    For options: e.g., ['S0', 'Volatility'].
            tff_input_raw_base_values (np.ndarray): 1D array of base values for these raw input factors.
            base_s0_for_convertible_tff (float, optional): Base S0 value if this TFF is for a
                                                           convertible bond and S0 is a dynamic factor.
                                                           This helps set the 'initial_stock_price' in
                                                           the static parameters for the TFF context.
                                                           Defaults to None.
        """
        self.product_type: str = None
        # This dict will hold parameters for ProductStaticBase.from_dict()
        self.product_static_params_for_worker: dict = {}
        self.pricer_config_for_worker: dict = {}

        self.convertible_tff_includes_s0_factor: bool = False # Specific to convertible TFF
        self.actual_rate_pillars: np.ndarray = None # Numeric rate pillars for QL bond pricers

        # For option TFF with engineered features
        self.option_tff_engineered_feature_names: list[str] = []
        self.normalization_means_for_tff_inputs: np.ndarray = None
        self.normalization_stds_for_tff_inputs: np.ndarray = None


        self.valuation_date_for_ql_settings_in_worker: date = pricer_template.product_static.valuation_date_py
        # tff_input_raw_factor_names are the names of the RAW factors the TFF is being built for
        # e.g., rate pillar numbers for bonds, or ['S0', 'Volatility'] for options before engineering
        self.tff_input_raw_factor_names: list[str] = [str(p) for p in tff_input_raw_factor_names]

        product_static_template = pricer_template.product_static

        # Prepare product_static_params_for_worker based on product type
        # This dictionary will be directly usable by the from_dict methods in worker.
        if isinstance(pricer_template, QuantLibBondPricer):
            bstatic = pricer_template.product_static # This is QuantLibBondStaticBase or derivative
            # Common bond parameters for reconstruction via from_dict
            self.product_static_params_for_worker = {
                # 'valuation_date' will be set by worker using valuation_date_for_ql_settings_in_worker
                'maturity_date': bstatic.maturity_date_py,
                'coupon_rate': bstatic.coupon_rate,
                'face_value': bstatic.face_value,
                'freq': bstatic.freq,
                'settlement_days': bstatic.settlement_days,
                # QL objects like calendar, day_count are not stored; from_dict will use defaults if None
            }

             # Determine product type and set actual_rate_pillars using the new helper
            if isinstance(bstatic, CallableBondStaticBase):
                self.product_type = 'callable'
                self.product_static_params_for_worker['product_type'] = 'callable'
                self.product_static_params_for_worker['call_dates'] = bstatic.call_dates_py
                self.product_static_params_for_worker['call_prices'] = bstatic.call_prices_py
                # Use the parser function
                self.actual_rate_pillars = _parse_numeric_pillars_from_factor_names(self.tff_input_raw_factor_names)

            elif isinstance(bstatic, ConvertibleBondStaticBase):
                self.product_type = 'convertible'
                self.product_static_params_for_worker['product_type'] = 'convertible'
                self.product_static_params_for_worker.update({
                    'issue_date': bstatic.issue_date_py,
                    'conversion_ratio': bstatic.conversion_ratio,
                    'dividend_yield': bstatic.dividend_yield,
                    'equity_volatility': bstatic.equity_volatility,
                    'credit_spread_value': bstatic.credit_spread_value,
                    'exercise_type': bstatic.exercise_type_str
                })
                if base_s0_for_convertible_tff is not None and 'S0' in self.tff_input_raw_factor_names:
                     self.convertible_tff_includes_s0_factor = True
                     self.product_static_params_for_worker['initial_stock_price'] = base_s0_for_convertible_tff
                else:
                    self.product_static_params_for_worker['initial_stock_price'] = bstatic.initial_stock_price
                # Use the parser function (it will ignore 'S0' if present in names)
                self.actual_rate_pillars = _parse_numeric_pillars_from_factor_names(self.tff_input_raw_factor_names)

            else: # Vanilla
                self.product_type = 'vanilla'
                self.product_static_params_for_worker['product_type'] = 'vanilla'
                # Use the parser function
                self.actual_rate_pillars = _parse_numeric_pillars_from_factor_names(self.tff_input_raw_factor_names)

            # Store actual_rate_pillars in the dict to be passed to worker for bond pricers
            if self.actual_rate_pillars is None: # Should be populated by now
                 raise ValueError("self.actual_rate_pillars was not set correctly for bond type.")
            self.product_static_params_for_worker['actual_rate_pillars'] = self.actual_rate_pillars

            self.pricer_config_for_worker['bond_pricer_config'] = {
                'method': pricer_template.method,
                'grid_steps': pricer_template.grid_steps,
                'convertible_engine_steps': pricer_template.convertible_engine_steps
            }

        elif isinstance(pricer_template, BlackScholesPricer):
            self.product_type = 'european_option'
            ostatic = pricer_template.product_static # This is EuropeanOptionStatic
            self.product_static_params_for_worker = {
                'product_type': 'european_option',
                'valuation_date': ostatic.valuation_date_py, # Worker will use its own
                'expiry_date': ostatic.expiry_date_py,
                'strike_price': ostatic.strike_price,
                'option_type': ostatic.option_type,
                # day_count_convention will use default in from_dict if not specified
            }
            self.pricer_config_for_worker['bs_pricer_config'] = {
                'risk_free_rate': pricer_template.risk_free_rate,
                'dividend_yield': pricer_template.dividend_yield
            }
            # For options, actual_rate_pillars is not directly used by BS pricer
            self.actual_rate_pillars = np.array([])
        else:
            raise TypeError("Unsupported pricer_template type for TFFCalibrate")

        self.tff_input_raw_base_values = tff_input_raw_base_values # Base values for raw factors

    def sample_and_fit(
        self, full_market_scenarios_raw: np.ndarray,
        n_train: int = 50, n_test: int = 20,
        random_seed: int = 0, sampling_method: str = 'sobol', parallel_workers: int = None,
        option_feature_order: int = 0,
        **price_kwargs
    ) -> tuple[TensorFunctionalForm, np.ndarray, np.ndarray, float, dict]:
        """
        Fits the TFF model by sampling scenarios, pricing them, and performing a least-squares regression.

        Args:
            full_market_scenarios_raw (np.ndarray): 2D array (N_total_scenarios, N_raw_factors)
                                                    of raw market factor scenarios (e.g., rates, or S0 & Vol).
                                                    These define the domain for sampling training points.
            n_train (int): Number of training samples to generate.
            n_test (int): Number of test samples to select from full_market_scenarios_raw.
            random_seed (int): Seed for reproducibility of sampling and test set selection.
            sampling_method (str): 'sobol', 'lhs', or 'uniform'.
            parallel_workers (int, optional): Number of worker processes for parallel pricing.
                                              If None, uses os.cpu_count(). If False or 0, runs sequentially.
            option_feature_order (int, optional): Order for option feature engineering.
                                                 If > 0 and product is 'european_option', features are engineered.
                                                 Defaults to 0 (no engineering).
            **price_kwargs: Additional keyword arguments for the pricer's `price` method
                            (e.g., g2_params for callable bonds).

        Returns:
            tuple:
                - model (TensorFunctionalForm): The fitted TFF model.
                - test_tff_inputs_raw_subset (np.ndarray): The raw input factors for the test set.
                - test_true_prices (np.ndarray): The true prices for the test set.
                - rmse (float): Root Mean Squared Error on the test set.
                - normalization_params (dict): Contains 'means', 'stds', 'engineered_feature_names', 'is_engineered'.
                                               Relevant for option TFFs with engineered features.
        """
        rng_np = np.random.default_rng(random_seed)
        num_raw_tff_factors = len(self.tff_input_raw_factor_names) # Number of S0, Vol, or Rate pillars

        if full_market_scenarios_raw.ndim!=2 or full_market_scenarios_raw.shape[1]!=num_raw_tff_factors:
            raise ValueError(f"full_market_scenarios_raw shape error. Expected (N,{num_raw_tff_factors}), got {full_market_scenarios_raw.shape}")

        domain_min,domain_max = np.min(full_market_scenarios_raw,axis=0),np.max(full_market_scenarios_raw,axis=0)
        # train_tff_inputs_raw are the raw factor values for the pricer (e.g., S0, Vol for option)
        train_tff_inputs_raw = None
        if sampling_method=='sobol': sampler=Sobol(d=num_raw_tff_factors,scramble=True,seed=random_seed); train_tff_inputs_raw=scale(sampler.random(n=n_train),domain_min,domain_max)
        elif sampling_method=='lhs': sampler_lhs=LatinHypercube(d=num_raw_tff_factors,centered=True,seed=random_seed); train_tff_inputs_raw=scale(sampler_lhs.random(n=n_train),domain_min,domain_max)
        elif sampling_method=='uniform': train_tff_inputs_raw=rng_np.uniform(low=domain_min,high=domain_max,size=(n_train,num_raw_tff_factors))
        else: raise ValueError(f"Unknown sampling: {sampling_method}.")

        # This dictionary is passed to the worker and contains all static info to reconstruct the product
        def_params_for_worker_pricing = self.product_static_params_for_worker

        train_prices = None
        if parallel_workers is not False and parallel_workers != 0:
            print(f"   Generating {n_train} training prices in parallel (workers={parallel_workers or 'default'})...")
            # Prepare arguments for each worker task
            args_for_tasks = [(def_params_for_worker_pricing, self.pricer_config_for_worker,
                               self.tff_input_raw_factor_names, # Names of raw factors for mapping
                               train_tff_inputs_raw[i], # Raw factor values for this scenario
                               self.valuation_date_for_ql_settings_in_worker,
                               price_kwargs)
                              for i in range(n_train)]
            with ProcessPoolExecutor(max_workers=parallel_workers) as executor:
                train_prices = np.array(list(executor.map(_price_one_scenario_for_tff, args_for_tasks)))
        else: # Sequential pricing
            print(f"   Generating {n_train} training prices sequentially...")
            ql_val_date_main_thread = ql.Date(self.valuation_date_for_ql_settings_in_worker.day,
                                              self.valuation_date_for_ql_settings_in_worker.month,
                                              self.valuation_date_for_ql_settings_in_worker.year)
            ql.Settings.instance().evaluationDate = ql_val_date_main_thread

            temp_prices_list = []
            for i in range(n_train):
                # Call the worker function directly for sequential execution
                args_for_seq = (def_params_for_worker_pricing, self.pricer_config_for_worker,
                                self.tff_input_raw_factor_names, train_tff_inputs_raw[i],
                                self.valuation_date_for_ql_settings_in_worker, price_kwargs)
                temp_prices_list.append(_price_one_scenario_for_tff(args_for_seq))
            train_prices = np.array(temp_prices_list)

        # Validate shape of train_prices
        if train_prices.ndim > 1 and train_prices.shape[0]!=n_train:
            train_prices = train_prices.squeeze() # Attempt to fix if it's (N,1) or (1,N)
        if train_prices.ndim == 0 and n_train == 1: # Convert scalar to 1-element array
            train_prices = np.array([train_prices])
        elif train_prices.ndim > 1 or (train_prices.ndim == 1 and train_prices.shape[0]!=n_train) :
             raise ValueError(f"Shape of train_prices {train_prices.shape} is not compatible with n_train {n_train}")

        # --- Feature Engineering & Normalization (primarily for options) ---
        tff_inputs_for_fitting = train_tff_inputs_raw
        normalization_params = {'means': None, 'stds': None,
                                'engineered_feature_names': self.tff_input_raw_factor_names, # Default before engineering
                                'is_engineered': False}

        if self.product_type == 'european_option' and option_feature_order > 0:
            print(f"   Engineering features for option TFF (order={option_feature_order})...")

            s0_factor_actual_name = None
            vol_factor_actual_name = None
            s0_idx = -1
            vol_idx = -1

            # self.tff_input_raw_factor_names for an option TFF is expected to be [full_s0_name, full_vol_name]
            # e.g., ["USD_STOCK_S0", "USD_STOCK_VOL"] as set up in main_demonstration.py
            if len(self.tff_input_raw_factor_names) == 2:
                # Attempt to identify S0 and Volatility factors by common suffixes from the two provided names
                for i, name in enumerate(self.tff_input_raw_factor_names):
                    if name.upper().endswith("_S0"):
                        s0_factor_actual_name = name
                        s0_idx = i
                    elif name.upper().endswith("_VOLATILITY") or name.upper().endswith("_VOL"):
                        vol_factor_actual_name = name
                        vol_idx = i

                # Validate that both were found and are distinct
                if s0_idx == -1 or vol_idx == -1 or s0_idx == vol_idx:
                    raise ValueError(
                        f"Could not reliably identify distinct S0 and Volatility factors for feature engineering "
                        f"from tff_input_raw_factor_names: {self.tff_input_raw_factor_names}. "
                        f"Expected two names, one ending with _S0 and one with _VOL or _VOLATILITY. "
                        f"Identified S0: '{s0_factor_actual_name}' at index {s0_idx}, "
                        f"Identified Vol: '{vol_factor_actual_name}' at index {vol_idx}."
                    )
            else:
                raise ValueError(
                    f"For option feature engineering, tff_input_raw_factor_names must contain exactly two "
                    f"elements (the S0 factor name and the Volatility factor name). "
                    f"Received: {self.tff_input_raw_factor_names}"
                )

            # train_tff_inputs_raw has columns corresponding to self.tff_input_raw_factor_names
            s0_train_raw = train_tff_inputs_raw[:, s0_idx]
            vol_train_raw = train_tff_inputs_raw[:, vol_idx]

            engineered_features_train, eng_feature_names = engineer_option_features(
                s0_train_raw, vol_train_raw, order=option_feature_order
            )
            tff_inputs_for_fitting, means, stds = normalize_features(engineered_features_train)

            normalization_params = {'means': means, 'stds': stds,
                                    'engineered_feature_names': eng_feature_names,
                                    'is_engineered': True}
        # ... (rest of the sample_and_fit method: D_tff_effective, X_train construction, lstsq, etc.)

        D_tff_effective = tff_inputs_for_fitting.shape[1] # Dimension of features actually fed to TFF

        # --- Least Squares Fitting for TFF Coefficients ---
        # Construct the design matrix X_train for y = X * coeffs
        # Each row of X_train corresponds to a training sample's (engineered & normalized) features.
        # Columns are [x1*x1, x1*x2, ..., xd*xd, x1, x2, ..., xd, 1]
        quadratic_terms_matrix = np.array([np.outer(s,s).flatten() for s in tff_inputs_for_fitting])
        X_train = np.hstack([
            quadratic_terms_matrix,          # (n_train, D_tff_effective^2) for A coefficients
            tff_inputs_for_fitting,          # (n_train, D_tff_effective) for b coefficients
            np.ones((n_train, 1))            # (n_train, 1) for c coefficient
        ])

        # Check for NaNs or Infs which can cause lstsq to fail
        if np.any(np.isnan(X_train)) or np.any(np.isinf(X_train)):
            raise ValueError("NaN or Inf found in training design matrix X_train after feature engineering/normalization.")
        if np.any(np.isnan(train_prices)) or np.any(np.isinf(train_prices)):
            raise ValueError("NaN or Inf found in train_prices (target values).")

        try:
            coeffs, _, _, _ = np.linalg.lstsq(X_train, train_prices, rcond=None)
        except np.linalg.LinAlgError as e:
            raise np.linalg.LinAlgError(f"Least squares fitting failed: {e}. Check for collinearity or issues in X_train or train_prices.")


        A_flat = coeffs[:D_tff_effective*D_tff_effective]
        A_matrix = A_flat.reshape(D_tff_effective, D_tff_effective)
        A_symmetric = 0.5 * (A_matrix + A_matrix.T) # Ensure A is symmetric

        b_vector = coeffs[D_tff_effective*D_tff_effective : D_tff_effective*D_tff_effective + D_tff_effective]
        c_scalar = coeffs[D_tff_effective*D_tff_effective + D_tff_effective]

        fitted_tff_model = TensorFunctionalForm(A_symmetric, b_vector, c_scalar)

        # --- Test Set Evaluation ---
        # Select a random subset from full_market_scenarios_raw for testing
        if n_test > full_market_scenarios_raw.shape[0]:
            print(f"Warning: n_test ({n_test}) > available scenarios ({full_market_scenarios_raw.shape[0]}). Using all for testing.")
            test_tff_inputs_raw_subset = full_market_scenarios_raw
        else:
            test_indices = rng_np.choice(full_market_scenarios_raw.shape[0], size=n_test, replace=False)
            test_tff_inputs_raw_subset = full_market_scenarios_raw[test_indices]

        # Get true prices for the test set using the original pricer
        ql_val_date_test_eval = ql.Date(self.valuation_date_for_ql_settings_in_worker.day,
                                     self.valuation_date_for_ql_settings_in_worker.month,
                                     self.valuation_date_for_ql_settings_in_worker.year)
        ql.Settings.instance().evaluationDate = ql_val_date_test_eval

        temp_test_prices_list = []
        for i in range(test_tff_inputs_raw_subset.shape[0]):
            args_for_test_seq = (def_params_for_worker_pricing, self.pricer_config_for_worker,
                                 self.tff_input_raw_factor_names, test_tff_inputs_raw_subset[i],
                                 self.valuation_date_for_ql_settings_in_worker, price_kwargs)
            temp_test_prices_list.append(_price_one_scenario_for_tff(args_for_test_seq))
        test_true_prices = np.array(temp_test_prices_list)

        # Prepare test inputs for TFF model (apply same engineering + normalization)
        # Prepare test inputs for TFF model (apply same engineering + normalization)
        test_inputs_for_tff_evaluation = test_tff_inputs_raw_subset

        # normalization_params was defined during the training phase feature engineering
        if self.product_type == 'european_option' and normalization_params.get('is_engineered', False):
            # This implies option_feature_order > 0 was used during training and normalization_params are set.

            s0_factor_actual_name_test = None
            vol_factor_actual_name_test = None
            s0_idx_test = -1
            vol_idx_test = -1

            # For an option TFF, self.tff_input_raw_factor_names is expected to be [full_s0_name, full_vol_name]
            # as set up in main_demonstration.py and passed to __init__.
            # These names correspond to the columns in test_tff_inputs_raw_subset.
            if len(self.tff_input_raw_factor_names) == 2:
                for i, name in enumerate(self.tff_input_raw_factor_names):
                    if name.upper().endswith("_S0"): # Using suffix matching
                        s0_factor_actual_name_test = name
                        s0_idx_test = i
                    elif name.upper().endswith("_VOLATILITY") or name.upper().endswith("_VOL"): # Using suffix matching
                        vol_factor_actual_name_test = name
                        vol_idx_test = i

                if s0_idx_test == -1 or vol_idx_test == -1 or s0_idx_test == vol_idx_test:
                    raise ValueError(
                        f"Could not reliably identify distinct S0 and Volatility factors for test set feature engineering "
                        f"from tff_input_raw_factor_names: {self.tff_input_raw_factor_names}. "
                        f"Expected two names, one ending with _S0 and one with _VOL or _VOLATILITY. "
                        f"Identified S0: '{s0_factor_actual_name_test}' at index {s0_idx_test}, "
                        f"Identified Vol: '{vol_factor_actual_name_test}' at index {vol_idx_test}."
                    )
            else:
                raise ValueError(
                    f"For option TFF test set evaluation (with engineered features), tff_input_raw_factor_names "
                    f"must contain exactly two elements (the S0 factor name and the Volatility factor name). "
                    f"Received: {self.tff_input_raw_factor_names}"
                )

            # Ensure test_tff_inputs_raw_subset has the correct number of columns
            if test_tff_inputs_raw_subset.shape[1] != len(self.tff_input_raw_factor_names):
                raise ValueError(
                    f"Mismatch between number of columns in test_tff_inputs_raw_subset ({test_tff_inputs_raw_subset.shape[1]}) "
                    f"and length of tff_input_raw_factor_names ({len(self.tff_input_raw_factor_names)}). "
                    f"This usually means test_tff_inputs_raw_subset was not selected correctly for this option TFF."
                )

            s0_test_raw = test_tff_inputs_raw_subset[:, s0_idx_test]
            vol_test_raw = test_tff_inputs_raw_subset[:, vol_idx_test]

            engineered_features_test, _ = engineer_option_features(
                s0_test_raw, vol_test_raw, order=option_feature_order # option_feature_order from method args
            )
            # Use normalization_params['means'] and ['stds'] calculated from the TRAINING set
            test_inputs_for_tff_evaluation, _, _ = normalize_features(
                engineered_features_test,
                normalization_params['means'],
                normalization_params['stds']
            )

        # Handle shapes for single test sample
        if test_true_prices.ndim > 1: test_true_prices = test_true_prices.squeeze()
        if test_true_prices.ndim == 0 and test_tff_inputs_raw_subset.shape[0] == 1:
             test_true_prices = np.array([float(test_true_prices)]) if test_true_prices.size > 0 else np.array([0.0])


        test_predicted_prices = fitted_tff_model(test_inputs_for_tff_evaluation)
        if test_predicted_prices.ndim > 1: test_predicted_prices = test_predicted_prices.squeeze()
        if test_predicted_prices.ndim == 0 and test_tff_inputs_raw_subset.shape[0] == 1:
            test_predicted_prices = np.array([float(test_predicted_prices)]) if test_predicted_prices.size > 0 else np.array([0.0])

        if test_true_prices.shape != test_predicted_prices.shape:
            raise ValueError(f"Shape mismatch for test prices: true {test_true_prices.shape}, predicted {test_predicted_prices.shape}")

        rmse = np.sqrt(np.mean((test_true_prices - test_predicted_prices)**2))

        return fitted_tff_model, test_tff_inputs_raw_subset, test_true_prices, rmse, normalization_params


In [ ]:
# utils.py
"""
Contains utility functions for the FastRiskDemo, such as generating
collections of bond definitions for portfolio testing.
"""
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta

def generate_bond_collections(
    num_bonds: int,
    valuation_date: date = date(2025, 1, 1),
    face_value: float = 100.0,
    seed: int = 0,
    conv_params: dict = None # Parameters for generating convertibles
) -> tuple[list[QuantLibBondStaticBase], list[CallableBondStaticBase], list[ConvertibleBondStaticBase]]:
    """
    Generates collections of random vanilla, callable, and convertible bond static definitions.

    Args:
        num_bonds (int): Number of bond instances of each type to generate.
        valuation_date (date, optional): Common valuation date for all bonds.
                                         Defaults to date(2025, 1, 1).
        face_value (float, optional): Default face value for bonds. Defaults to 100.0.
        seed (int, optional): Random seed for reproducibility. Defaults to 0.
        conv_params (dict, optional): Dictionary of parameters for generating convertible bonds.
                                      Keys can include 'conversion_ratio', 'dividend_yield',
                                      'equity_volatility', 'initial_stock_price',
                                      'credit_spread_value', 'exercise_type'.
                                      Defaults are used if not provided.

    Returns:
        tuple[list, list, list]:
            A tuple containing:
            - A list of QuantLibBondStaticBase instances (vanilla bonds).
            - A list of CallableBondStaticBase instances (callable bonds).
            - A list of ConvertibleBondStaticBase instances (convertible bonds).
    """
    rng = np.random.default_rng(seed)
    vanilla_bonds = []
    callable_bonds = []
    convertible_bonds = []

    # Default parameters for convertible bond generation
    default_conv_params = {
        'conversion_ratio': 20.0,        # Example: 20 shares per bond of 100 face value
        'dividend_yield': 0.01,         # 1% continuous dividend yield
        'equity_volatility': 0.25,      # 25% annual volatility
        'initial_stock_price': 100.0,   # Base stock price
        'credit_spread_value': 0.015,   # 150 bps credit spread
        'exercise_type': 'EuropeanAtMaturity'
    }
    # Update defaults with any user-provided conv_params
    current_conv_params = default_conv_params.copy()
    if conv_params is not None:
        current_conv_params.update(conv_params)

    for i in range(num_bonds):
        # Common random parameters for each bond in the iteration
        years_to_maturity = int(rng.integers(3, 11)) # Ensure bonds have some life, min 3 years
        maturity_d = valuation_date + relativedelta(years=years_to_maturity)

        # Use different coupon ranges for different bond types for more realism
        van_call_coupon = float(rng.uniform(0.02, 0.06)) # 2% to 6% for vanilla/callable
        conv_coupon = float(rng.uniform(0.01, 0.04)) # 1% to 4% for convertibles (often lower)

        coupon_freq = int(rng.choice([1, 2]))    # Annual or Semi-annual

        # --- Vanilla Bond ---
        # Parameters for QuantLibBondStaticBase.from_dict
        vanilla_params_dict = {
            'valuation_date': valuation_date,
            'maturity_date': maturity_d,
            'coupon_rate': van_call_coupon,
            'face_value': face_value,
            'freq': coupon_freq,
            'settlement_days': 0
        }
        vanilla_bonds.append(QuantLibBondStaticBase.from_dict(vanilla_params_dict))

        # --- Callable Bond ---
        call_dates_list_py = []
        call_prices_list_py = []
        if years_to_maturity > 2: # Ensure some non-call period for callable bonds
            # Generate 1 to 3 call dates for variety
            num_calls = int(rng.integers(1, min(4, years_to_maturity - 1)))
            possible_call_years_offsets = list(range(1, years_to_maturity)) # Call can be from year 1 to year T-1

            if num_calls > 0 and len(possible_call_years_offsets) >= num_calls:
                chosen_call_years_offsets = sorted(rng.choice(possible_call_years_offsets, size=num_calls, replace=False))
                for year_offset in chosen_call_years_offsets:
                    call_d = valuation_date + relativedelta(years=int(year_offset))
                    if call_d < maturity_d: # Ensure call date is before maturity
                        call_dates_list_py.append(call_d)
                        # Call price typically at par or a slight premium
                        call_prices_list_py.append(float(face_value + rng.uniform(0.0, 3.0)))

        if call_dates_list_py and call_prices_list_py: # Only if valid call dates were generated
            callable_params_dict = {
                'valuation_date': valuation_date, 'maturity_date': maturity_d,
                'coupon_rate': van_call_coupon, 'face_value': face_value,
                'freq': coupon_freq, 'settlement_days': 0,
                'call_dates': call_dates_list_py, 'call_prices': call_prices_list_py
            }
            callable_bonds.append(CallableBondStaticBase.from_dict(callable_params_dict))

        # --- Convertible Bond ---
        # Issue date for convertible, can be different from valuation date.
        issue_offset_years = int(rng.integers(0, min(3, years_to_maturity - 1)))
        issue_date_convertible = valuation_date - relativedelta(years=issue_offset_years)
        # Ensure issue date is not after valuation date for this setup
        if issue_date_convertible > valuation_date:
            issue_date_convertible = valuation_date

        # Randomize some convertible parameters slightly for variety around the provided base
        s0_rand = current_conv_params['initial_stock_price'] * rng.uniform(0.8, 1.2)
        vol_rand = current_conv_params['equity_volatility'] * rng.uniform(0.7, 1.3)
        div_rand = max(0, current_conv_params['dividend_yield'] * rng.uniform(0.5, 1.5))
        cs_rand = max(0.001, current_conv_params['credit_spread_value'] * rng.uniform(0.5, 2.0))
        cr_rand = current_conv_params['conversion_ratio'] * rng.uniform(0.9, 1.1)

        convertible_params_dict = {
            'valuation_date': valuation_date,
            'issue_date': issue_date_convertible,
            'maturity_date': maturity_d,
            'coupon_rate': conv_coupon,
            'conversion_ratio': cr_rand,
            'dividend_yield': div_rand,
            'equity_volatility': vol_rand,
            'initial_stock_price': s0_rand,
            'credit_spread_value': cs_rand,
            'face_value': face_value,
            'freq': coupon_freq,
            'settlement_days': 0,
            'exercise_type': current_conv_params['exercise_type']
        }
        convertible_bonds.append(ConvertibleBondStaticBase.from_dict(convertible_params_dict))

    return vanilla_bonds, callable_bonds, convertible_bonds


In [ ]:
# scenario_generator.py
"""
Contains classes for generating market scenarios for different risk factors.
"""
import numpy as np
import abc

class ScenarioGeneratorBase(abc.ABC):
    """
    Abstract base class for scenario generators.
    """
    @abc.abstractmethod
    def generate_scenarios(self, num_scenarios: int) -> tuple[np.ndarray, list[str]]:
        """
        Generates market scenarios.

        Args:
            num_scenarios (int): The number of scenarios to generate.

        Returns:
            tuple[np.ndarray, list[str]]:
                - A 2D NumPy array where rows are scenarios and columns are risk factors.
                - A list of strings representing the names of the risk factors (columns).
        """
        pass

class SimpleRandomScenarioGenerator(ScenarioGeneratorBase):
    """
    Generates scenarios using simple random normal shocks around base values
    for rates, and uniform or normal shocks for S0 and Volatility.

    Args:
        base_rates (np.ndarray, optional): 1D array of base interest rates for pillars.
        rate_pillar_names (list[str], optional): Names for rate pillars. If None and base_rates
                                                 is provided, names like "rate_pillar_0" are generated.
        rate_shock_std_dev (float, optional): Standard deviation for normal rate shocks (absolute).
                                              Defaults to 0.0010 (10 bps).
        base_s0 (float, optional): Base stock price. If None, S0 scenarios are not generated.
        s0_shock_type (str, optional): Type of shock for S0: 'normal' or 'uniform'.
                                       Defaults to 'normal'.
        s0_shock_param (float, optional): Parameter for S0 shock.
                                          If 'normal', this is the standard deviation as a percentage
                                          of base_s0 (e.g., 0.10 for 10% std dev).
                                          If 'uniform', this is the half-width of the uniform range
                                          as a percentage of base_s0 (e.g., 0.10 for base_s0 +/- 10%).
                                          Defaults to 0.10.
        base_vol (float, optional): Base volatility. If None, volatility scenarios are not generated.
        vol_shock_type (str, optional): Type of shock for volatility: 'normal' or 'uniform'.
                                        Defaults to 'normal'.
        vol_shock_param (float, optional): Parameter for volatility shock, similar to s0_shock_param.
                                           Defaults to 0.05.
        random_seed (int, optional): Seed for NumPy's random number generator for reproducibility.
                                     Defaults to None (no fixed seed).
    """
    def __init__(self,
      # Rate factor configurations
      base_rates_map: dict[str, float] = None, # e.g., {"USD_SOFR_0.25Y": 0.02, "EUR_EURIBOR6M_0.5Y": 0.01}
      rate_factor_shock_std_dev_map: dict[str, float] = None, # e.g., {"USD_SOFR_0.25Y": 0.0010} or a default
                                                            # If None, a global default can be used.
      # S0 factor configurations
      base_s0_map: dict[str, float] = None,      # e.g., {"USD_EQUITY_AAPL_S0": 150.0}
      s0_shock_config_map: dict[str, tuple[str, float]] = None, # e.g., {"USD_EQUITY_AAPL_S0": ('normal', 0.10)}
                                                              # shock_type, shock_param
      # Volatility factor configurations
      base_vol_map: dict[str, float] = None,     # e.g., {"USD_EQUITY_AAPL_VOL": 0.20}
      vol_shock_config_map: dict[str, tuple[str, float]] = None, # e.g., {"USD_EQUITY_AAPL_VOL": ('uniform', 0.05)}

      default_rate_shock_std_dev: float = 0.0010,
      default_s0_shock_config: tuple[str, float] = ('normal', 0.10),
      default_vol_shock_config: tuple[str, float] = ('normal', 0.05),
      random_seed: int = None):

      self.base_rates_map = base_rates_map if base_rates_map is not None else {}
      self.rate_factor_shock_std_dev_map = rate_factor_shock_std_dev_map if rate_factor_shock_std_dev_map is not None else {}

      self.base_s0_map = base_s0_map if base_s0_map is not None else {}
      self.s0_shock_config_map = s0_shock_config_map if s0_shock_config_map is not None else {}

      self.base_vol_map = base_vol_map if base_vol_map is not None else {}
      self.vol_shock_config_map = vol_shock_config_map if vol_shock_config_map is not None else {}

      self.default_rate_shock_std_dev = default_rate_shock_std_dev
      self.default_s0_shock_config = default_s0_shock_config
      self.default_vol_shock_config = default_vol_shock_config

      self.rng = np.random.default_rng(random_seed)

      # Consolidate all unique factor names that will be generated
      self.factor_names_ordered = sorted(list(
          set(self.base_rates_map.keys()) |
          set(self.base_s0_map.keys()) |
          set(self.base_vol_map.keys())
      ))
      if not self.factor_names_ordered and (self.base_rates_map or self.base_s0_map or self.base_vol_map) :
          # This might happen if maps are passed but keys are somehow not strings, though type hints suggest strings
          raise ValueError("Could not determine ordered factor names. Ensure keys in maps are strings.")

    def generate_scenarios(self, num_scenarios: int) -> tuple[np.ndarray, list[str]]:
        """
        Generates the market scenarios.

        Args:
            num_scenarios (int): The number of scenarios to generate.

        Returns:
            tuple[np.ndarray, list[str]]:
                - scenarios (np.ndarray): 2D array (num_scenarios, num_factors).
                - factor_names (list[str]): List of factor names corresponding to columns.
        """
        if not self.factor_names_ordered:
            return np.array([]).reshape(num_scenarios, 0), []

        all_scenario_columns = []

        for factor_name in self.factor_names_ordered:
            factor_column = np.zeros(num_scenarios)

            if factor_name in self.base_rates_map:
                base_value = self.base_rates_map[factor_name]
                shock_std_dev = self.rate_factor_shock_std_dev_map.get(factor_name, self.default_rate_shock_std_dev)
                shocks = self.rng.normal(loc=0.0, scale=shock_std_dev, size=num_scenarios)
                factor_column = base_value + shocks

            elif factor_name in self.base_s0_map:
                base_value = self.base_s0_map[factor_name]
                shock_type, shock_param = self.s0_shock_config_map.get(factor_name, self.default_s0_shock_config)

                if shock_type.lower() == 'normal':
                    actual_std_dev = shock_param * base_value if shock_param <= 1.0 else shock_param # allow abs or relative
                    shocks = self.rng.normal(loc=0.0, scale=actual_std_dev, size=num_scenarios)
                    factor_column = base_value + shocks
                elif shock_type.lower() == 'uniform':
                    half_width = shock_param * base_value if shock_param <= 1.0 else shock_param
                    min_val = base_value - half_width
                    max_val = base_value + half_width
                    factor_column = self.rng.uniform(min_val, max_val, size=num_scenarios)
                else:
                    raise ValueError(f"Unsupported shock_type: {shock_type} for S0 factor {factor_name}")
                factor_column = np.maximum(factor_column, 1e-6) # Ensure S0 is positive

            elif factor_name in self.base_vol_map:
                base_value = self.base_vol_map[factor_name]
                shock_type, shock_param = self.vol_shock_config_map.get(factor_name, self.default_vol_shock_config)

                if shock_type.lower() == 'normal':
                    actual_std_dev = shock_param * base_value if shock_param <= 1.0 else shock_param
                    shocks = self.rng.normal(loc=0.0, scale=actual_std_dev, size=num_scenarios)
                    factor_column = base_value + shocks
                elif shock_type.lower() == 'uniform':
                    half_width = shock_param * base_value if shock_param <= 1.0 else shock_param
                    min_val = base_value - half_width
                    max_val = base_value + half_width
                    factor_column = self.rng.uniform(min_val, max_val, size=num_scenarios)
                else:
                    raise ValueError(f"Unsupported shock_type: {shock_type} for Vol factor {factor_name}")
                factor_column = np.maximum(factor_column, 1e-6) # Ensure vol is positive

            else:
                # Should not happen if factor_names_ordered is built correctly from map keys
                raise ValueError(f"Factor name {factor_name} not found in any configuration map.")

            all_scenario_columns.append(factor_column[:, np.newaxis]) # Ensure it's a column vector

        return np.hstack(all_scenario_columns), self.factor_names_ordered


In [ ]:
# portfolio.py
"""
Contains classes for defining and analyzing portfolios of financial instruments.
The Portfolio class allows pricing using either TFF models (retrieved from a cache)
or full pricers.
"""
import numpy as np
import abc

class PortfolioBase(abc.ABC):
    """
    Abstract base class for a portfolio of financial instruments.
    """
    def __init__(self):
        # Stores details for each *position* in the portfolio.
        # Multiple positions can refer to the same underlying instrument_id if TFFs are cached.
        self.positions: list[dict] = []

    @abc.abstractmethod
    def add_position(self, *args, **kwargs):
        """Adds a position (instrument holding) to the portfolio."""
        pass

    @abc.abstractmethod
    def price_portfolio(self,
                        raw_market_scenarios: np.ndarray,
                        scenario_factor_names: list[str],
                        portfolio_rate_pillar_times: np.ndarray = None # For bond pricers
                        ) -> np.ndarray:
        """
        Prices all instruments in the portfolio for given market scenarios.

        Args:
            raw_market_scenarios (np.ndarray): 2D array of market scenarios
                                               (N_scenarios, N_total_market_factors).
            scenario_factor_names (list[str]): Names of the columns in raw_market_scenarios.
            portfolio_rate_pillar_times (np.ndarray, optional): 1D array of rate pillar times.
                                                               Required if portfolio contains bonds
                                                               priced with full QuantLibBondPricer.
        Returns:
            np.ndarray: 1D array of aggregated portfolio prices for each scenario (N_scenarios,).
        """
        pass

class Portfolio(PortfolioBase):
    """
    A portfolio where each instrument can be priced using either a pre-fitted
    Tensor Functional Form (TFF) model (retrieved from a cache) or its original full pricer.
    """
    def __init__(self):
        super().__init__()
        self.tff_model_cache: dict = {}
        # Cache structure:
        # { instrument_id: {
        #     'tff_model': TensorFunctionalForm_object,
        #     'raw_tff_input_names': list_of_names, # Raw factors TFF was trained on
        #     'normalization_params': dict_of_norm_params, # Includes engineered_feature_names
        #     'option_feature_order': int
        #   }, ...
        # }

    def cache_tff_model(self,
                        instrument_id: str,
                        tff_model: TensorFunctionalForm,
                        raw_tff_input_names: list[str],
                        normalization_params: dict,
                        option_feature_order: int = 0):
        """
        Explicitly caches a fitted TFF model and its associated parameters.

        Args:
            instrument_id (str): Unique ID for the instrument type this TFF represents.
            tff_model (TensorFunctionalForm): The pre-fitted TFF model.
            raw_tff_input_names (list[str]): Names of raw market factors TFF is based on.
            normalization_params (dict): Normalization params from TFF calibration.
                                         Expected keys: 'means', 'stds', 'engineered_feature_names', 'is_engineered'.
            option_feature_order (int, optional): Order of feature engineering if option TFF.
        """
        if not instrument_id:
            raise ValueError("instrument_id must be provided for caching TFF model.")
        if not isinstance(tff_model, TensorFunctionalForm):
            raise TypeError("tff_model must be an instance of TensorFunctionalForm.")
        if raw_tff_input_names is None or not isinstance(raw_tff_input_names, list):
            raise ValueError("raw_tff_input_names (list of strings) must be provided for caching TFF model.")
        if normalization_params is None or not isinstance(normalization_params, dict):
            raise ValueError("normalization_params (dict) must be provided for caching TFF model.")

        self.tff_model_cache[instrument_id] = {
            'tff_model': tff_model,
            'raw_tff_input_names': raw_tff_input_names,
            'normalization_params': normalization_params,
            'option_feature_order': option_feature_order
        }

    def add_position(self,
                       instrument_id: str,
                       product_static: ProductStaticBase,
                       num_holdings: int = 1,
                       pricing_engine_type: str = 'tff',
                       direct_tff_config: dict = None,
                       full_pricer_instance: PricerBase = None,
                       full_pricer_kwargs: dict = None):
        """
        Adds a position (an instrument holding) to the portfolio.
        If pricing_engine_type is 'tff', it retrieves the TFF model from the cache
        using the instrument_id. The TFF model must have been cached previously using `cache_tff_model`.

        Args:
            instrument_id (str): Unique ID for this instrument type. Used for TFF caching/lookup.
            product_static (ProductStaticBase): Static definition of the product for this position.
            num_holdings (int, optional): Number of units. Defaults to 1.
            pricing_engine_type (str, optional): 'tff' or 'full'. Defaults to 'tff'.
            full_pricer_instance (PricerBase, optional): A full pricer instance (required if type is 'full').
            full_pricer_kwargs (dict, optional): Keyword arguments for the full pricer's price method.
        """
        if not isinstance(num_holdings, int) or num_holdings <= 0:
                raise ValueError("num_holdings must be a positive integer.")
        if not instrument_id:
            raise ValueError("instrument_id must be provided for the position.")

        position_detail = {
            'instrument_id': instrument_id,
            'product_static': product_static,
            'num_holdings': num_holdings,
            'engine_type': pricing_engine_type.lower()
        }

        if position_detail['engine_type'] == 'tff':
            if direct_tff_config is not None:
                # A TFF model and its configuration are being provided directly as a dictionary
                if not isinstance(direct_tff_config, dict):
                    raise TypeError("direct_tff_config must be a dictionary.")

                model_dict = direct_tff_config.get('model_dict')
                raw_names = direct_tff_config.get('raw_input_names')
                norm_params = direct_tff_config.get('normalization_params')

                # Default option_feature_order to 0 if not explicitly in config
                opt_order = direct_tff_config.get('option_feature_order', 0)

                if model_dict is None or not isinstance(model_dict, dict):
                    raise ValueError("direct_tff_config is missing 'model_dict' or it's not a dictionary.")
                if raw_names is None or not isinstance(raw_names, list):
                    raise ValueError("direct_tff_config is missing 'raw_input_names' or it's not a list.")
                if norm_params is None or not isinstance(norm_params, dict):
                    raise ValueError("direct_tff_config is missing 'normalization_params' or it's not a dictionary.")

                # Deserialize the TFF model itself from model_dict
                try:
                    tff_model_instance = TensorFunctionalForm.from_dict(model_dict)
                except Exception as e:
                    raise ValueError(f"Failed to deserialize TFF model from direct_tff_config['model_dict']: {e}")

                position_detail['pricer_engine'] = tff_model_instance
                position_detail['raw_tff_input_names'] = raw_names
                position_detail['normalization_params'] = norm_params
                position_detail['option_feature_order'] = opt_order

            else:
                # No direct TFF config provided, so use the cache via instrument_id
                if instrument_id not in self.tff_model_cache:
                    raise ValueError(
                        f"TFF model for instrument_id '{instrument_id}' not found in cache. "
                        "Either fit and cache it first, or provide its configuration via 'direct_tff_config'."
                    )

                cached_data = self.tff_model_cache[instrument_id]
                position_detail['pricer_engine'] = cached_data['tff_model']
                position_detail['raw_tff_input_names'] = cached_data['raw_tff_input_names']
                position_detail['normalization_params'] = cached_data['normalization_params']
                position_detail['option_feature_order'] = cached_data['option_feature_order']

        elif position_detail['engine_type'] == 'full':
            if not isinstance(full_pricer_instance, PricerBase):
                raise TypeError("full_pricer_instance must be an instance of PricerBase if engine_type is 'full'.")
            position_detail['pricer_engine'] = full_pricer_instance
            position_detail['full_pricer_kwargs'] = full_pricer_kwargs or {}
        else:
            raise ValueError(f"Unsupported pricing_engine_type: {pricing_engine_type}. Choose 'tff' or 'full'.")

        self.positions.append(position_detail)


    def price_portfolio(self,
                        raw_market_scenarios: np.ndarray,
                        scenario_factor_names: list[str],
                        portfolio_rate_pillar_times: np.ndarray = None
                        ) -> np.ndarray:
        """
        Prices all positions in the portfolio for given market scenarios.

        Args:
            raw_market_scenarios (np.ndarray): 2D array of raw market scenarios
                                               (N_scenarios, N_total_market_factors).
            scenario_factor_names (list[str]): Names of the columns in raw_market_scenarios.
            portfolio_rate_pillar_times (np.ndarray, optional): 1D array of rate pillar times (numeric tenors).
                                                               Required if portfolio contains bonds
                                                               priced with full QuantLibBondPricer or FastBondPricer.

        Returns:
            np.ndarray: 1D array of aggregated portfolio prices for each scenario (N_scenarios,).
        """
        if not self.positions:
            return np.array([])

        num_scenarios = raw_market_scenarios.shape[0]
        portfolio_prices_per_scenario = np.zeros(num_scenarios, dtype=float)

        for position_detail in self.positions:
            pricer_engine = position_detail['pricer_engine']
            engine_type = position_detail['engine_type']
            num_holdings = position_detail['num_holdings']

            instrument_prices_this_instrument = np.zeros(num_scenarios, dtype=float)

            if engine_type == 'tff':
                tff_model: TensorFunctionalForm = pricer_engine
                # These are the names of the RAW factors the TFF was originally trained on.
                raw_tff_input_factor_names_for_this_tff = position_detail['raw_tff_input_names']
                norm_params = position_detail['normalization_params']
                opt_feat_order = position_detail['option_feature_order']

                # Select the relevant columns from the global raw_market_scenarios
                try:
                    indices_of_raw_factors_in_global_scenarios = [scenario_factor_names.index(name) for name in raw_tff_input_factor_names_for_this_tff]
                except ValueError as e:
                    raise ValueError(f"A TFF input name in {raw_tff_input_factor_names_for_this_tff} not found in scenario_factor_names {scenario_factor_names} for instrument_id '{position_detail['instrument_id']}'. Error: {e}")

                current_raw_inputs_for_tff = raw_market_scenarios[:, indices_of_raw_factors_in_global_scenarios]

                inputs_for_tff_evaluation = current_raw_inputs_for_tff # Default

                # opt_feat_order should be available from position_detail if it's an option TFF
                opt_feat_order = position_detail.get('option_feature_order', 0)

                if isinstance(position_detail['product_static'], EuropeanOptionStatic) and \
                   norm_params.get('is_engineered', False): # Check if TFF was trained with engineered features

                    s0_factor_actual_name_port = None
                    vol_factor_actual_name_port = None
                    # These indices are relative to raw_tff_input_factor_names_for_this_tff
                    # and thus to the columns of current_raw_inputs_for_tff
                    s0_idx_in_tff_inputs = -1
                    vol_idx_in_tff_inputs = -1

                    # For an option TFF, raw_tff_input_factor_names_for_this_tff is expected to be [full_s0_name, full_vol_name]
                    if len(raw_tff_input_factor_names_for_this_tff) == 2:
                        for i, name in enumerate(raw_tff_input_factor_names_for_this_tff):
                            if name.upper().endswith("_S0"): # Using suffix matching
                                s0_factor_actual_name_port = name
                                s0_idx_in_tff_inputs = i
                            elif name.upper().endswith("_VOLATILITY") or name.upper().endswith("_VOL"): # Using suffix matching
                                vol_factor_actual_name_port = name
                                vol_idx_in_tff_inputs = i

                        if s0_idx_in_tff_inputs == -1 or vol_idx_in_tff_inputs == -1 or s0_idx_in_tff_inputs == vol_idx_in_tff_inputs:
                            raise ValueError(
                                f"Portfolio pricing: Could not identify distinct S0 and Volatility factors for option "
                                f"'{position_detail['instrument_id']}' from its TFF input names: "
                                f"{raw_tff_input_factor_names_for_this_tff}."
                            )
                    else:
                        raise ValueError(
                            f"Portfolio pricing: Option TFF for '{position_detail['instrument_id']}' with engineered features "
                            f"expects 2 raw input factors (S0, Vol), but "
                            f"TFF was trained on {len(raw_tff_input_factor_names_for_this_tff)} names: "
                            f"{raw_tff_input_factor_names_for_this_tff}."
                        )

                    # current_raw_inputs_for_tff columns are ordered as per raw_tff_input_factor_names_for_this_tff
                    s0_scenarios_raw = current_raw_inputs_for_tff[:, s0_idx_in_tff_inputs]
                    vol_scenarios_raw = current_raw_inputs_for_tff[:, vol_idx_in_tff_inputs]

                    engineered_features_test, _ = engineer_option_features( # Changed name for clarity
                        s0_scenarios_raw, vol_scenarios_raw, order=opt_feat_order
                    )
                    # Use normalization_params['means'] and ['stds'] from the cached TFF model data
                    inputs_for_tff_evaluation, _, _ = normalize_features(
                        engineered_features_test, # Changed name
                        norm_params['means'],
                        norm_params['stds']
                    )

                instrument_prices_this_instrument = tff_model(inputs_for_tff_evaluation)

            elif engine_type == 'full':
                full_pricer: PricerBase = pricer_engine
                pricer_kwargs = position_detail.get('full_pricer_kwargs', {})

                if isinstance(full_pricer, (QuantLibBondPricer, FastBondPricer)):
                    if portfolio_rate_pillar_times is None:
                        raise ValueError("portfolio_rate_pillar_times must be provided for full QL/Fast bond pricing in portfolio.")

                    num_rate_pillars = len(portfolio_rate_pillar_times)
                    # Select rate columns from raw_market_scenarios
                    rate_indices = []
                    try:
                        # Attempt to find by name if portfolio_rate_pillar_times contains names (should be numeric tenors)
                        # For simplicity, assume scenario_factor_names aligns with rate_pillar_names for the rate part
                        rate_indices = [scenario_factor_names.index(name) for name in portfolio_rate_pillar_times if isinstance(name, str) and name in scenario_factor_names]
                        if len(rate_indices) != num_rate_pillars: # Fallback if names don't match
                             rate_indices = list(range(num_rate_pillars))
                    except (ValueError, IndexError, TypeError):
                        # Fallback if portfolio_rate_pillar_times are not strings or not found
                        rate_indices = list(range(num_rate_pillars))

                    market_data_for_bond_pricer = raw_market_scenarios[:, rate_indices]

                    if isinstance(full_pricer, QuantLibBondPricer) and full_pricer.is_convertible_bond_type and 'S0' in scenario_factor_names:
                        s0_col_idx = scenario_factor_names.index('S0')
                        market_data_for_bond_pricer = np.hstack((
                            market_data_for_bond_pricer,
                            raw_market_scenarios[:, s0_col_idx, np.newaxis]
                        ))

                    instrument_prices_this_instrument = full_pricer.price(
                        pillar_times=portfolio_rate_pillar_times, # These are the numeric tenors
                        market_scenario_data=market_data_for_bond_pricer,
                        **pricer_kwargs
                    )
                elif isinstance(full_pricer, BlackScholesPricer):
                    s0_idx = scenario_factor_names.index('S0')
                    vol_idx = scenario_factor_names.index('Volatility')
                    s0_scens = raw_market_scenarios[:, s0_idx]
                    vol_scens = raw_market_scenarios[:, vol_idx]
                    instrument_prices_this_instrument = full_pricer.price(
                        stock_price=s0_scens,
                        volatility=vol_scens,
                        **pricer_kwargs
                    )
                else:
                    raise TypeError(f"Unsupported full pricer type in portfolio: {type(full_pricer)} for instrument_id '{position_detail['instrument_id']}'")
            else:
                raise ValueError(f"Unknown engine_type: {engine_type} for instrument_id '{position_detail['instrument_id']}'")

            # Ensure consistent shapes for aggregation
            if instrument_prices_this_instrument.ndim == 0:
                instrument_prices_this_instrument = np.full(num_scenarios, float(instrument_prices_this_instrument))
            elif len(instrument_prices_this_instrument) != num_scenarios:
                 instrument_prices_this_instrument = instrument_prices_this_instrument.flatten()
                 if len(instrument_prices_this_instrument) != num_scenarios:
                    raise ValueError(f"Price array shape mismatch for instrument '{position_detail['instrument_id']}'. Expected ({num_scenarios},), got {instrument_prices_this_instrument.shape}")

            portfolio_prices_per_scenario += instrument_prices_this_instrument * num_holdings

        return portfolio_prices_per_scenario


In [ ]:
# main_demonstration.py
"""
Main script to demonstrate bond and option pricing, Tensor Functional Form (TFF)
approximation, performance comparisons, and portfolio pricing using a modular structure.
Demonstrates TFF caching and portfolio definition, now with currency-specific factors.
"""
import QuantLib as ql
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
import time
import os

# Assuming other necessary modules are in the same directory or PYTHONPATH
# from product_definitions import (ProductStaticBase, QuantLibBondStaticBase,
#                                  CallableBondStaticBase, ConvertibleBondStaticBase,
#                                  EuropeanOptionStatic)
# from pricers import (PricerBase, FastBondPricer, QuantLibBondPricer,
#                      BlackScholesPricer)
# from g2_model import G2Calibrator
# from tff_approximator import (TensorFunctionalForm, TensorFunctionalFormCalibrate,
#                               engineer_option_features, normalize_features)
# from scenario_generator import SimpleRandomScenarioGenerator
# from portfolio import Portfolio
# from utils import generate_bond_collections


def run_demonstration(
    enable_parallel_tff_fitting: bool = True,
    g2_model_grid_steps_param: int = 32,
    convertible_binomial_steps_param: int = 50,
    option_tff_factors_s0_vol: bool = True,
    bond_tff_for_convertible_includes_s0: bool = True,
    option_feature_eng_order: int = 2,
    use_hardcoded_g2_params: bool = True
    ):
    """
    Runs the full demonstration with currency-specific factor handling.
    """
    print(f"--- FastRiskDemo (Parallel TFF: {enable_parallel_tff_fitting}, G2 Grid: {g2_model_grid_steps_param}, "
          f"Conv Grid: {convertible_binomial_steps_param}, OptTFF(S0,Vol): {option_tff_factors_s0_vol}, "
          f"ConvBondTFF(Rates,S0): {bond_tff_for_convertible_includes_s0}, "
          f"OptFeatOrder: {option_feature_eng_order}, G2Calib: {'Disabled' if use_hardcoded_g2_params else 'Enabled'}) ---")

    # --- Configuration Parameters ---
    G2_MODEL_GRID_STEPS = g2_model_grid_steps_param
    CONV_ENGINE_STEPS = convertible_binomial_steps_param
    G2_CALIBRATION_ENGINE_STEPS = 30

    SIMULATION_VARIANCE_FACTOR_RATES = 0.001  # Default shock for rates
    SIMULATION_VARIANCE_FACTOR_S0_PERCENT = 0.20 # Default shock for S0 as percentage
    SIMULATION_VARIANCE_FACTOR_VOL_PERCENT = 0.30 # Default shock for Vol as percentage

    N_SCENARIOS_FOR_TFF_DOMAIN = 2000
    N_SCENARIOS_FOR_BENCHMARK_SUBSET_PERCENT = 0.01 # Use 1% of scenarios for QL timing

    # --- Dates and Basic Product Parameters ---
    val_d = date(2025, 5, 18)
    issue_date_conv_ex = date(2024, 5, 18)
    option_expiry_ex = val_d + relativedelta(years=1)

    van_mat = val_d + relativedelta(years=5)
    call_mat = val_d + relativedelta(years=5)
    conv_mat = issue_date_conv_ex + relativedelta(years=5) # Maturity relative to its issue date

    coupon_rate_generic = 0.032
    face_value_generic = 100.0

    call_dates_example = [val_d + relativedelta(years=y) for y in [2,3,4]]
    call_prices_example = [100.0, 100.0, 100.0]

    CONV_S0_BASE = 100.0
    CONV_VOL_BASE = 0.25
    CONV_DIV_YIELD = 0.01
    CONV_CREDIT_SPREAD = 0.015
    CONV_CONVERSION_RATIO = 20.0

    OPT_STRIKE_PRICE = 105.0
    OPT_TYPE_STR = 'call'
    OPT_BS_RISK_FREE_RATE = 0.025 # This rate is specific to BS model, not a scenario factor here
    OPT_BS_DIVIDEND_YIELD = 0.01  # This is specific to BS model

    # --- Define Currency and Index Stubs to be used for this demonstration ---
    DEMO_CURRENCY = "USD"
    DEMO_RATE_INDEX_STUB = "IR"  # Generic for Interest Rate, could be "SOFR", "LIBOR" etc.
    DEMO_EQUITY_STUB = "STOCK"   # Generic for the equity underlying convertibles/options

    # --- 1. Defining Product Static Objects (now with currency/index) ---
    print(f"\n1. Defining Product Static Objects (Currency: {DEMO_CURRENCY})...")

    vanilla_params_dict = {
        'valuation_date': val_d, 'maturity_date': van_mat,
        'coupon_rate': coupon_rate_generic, 'face_value': face_value_generic,
        'settlement_days': 0, 'freq': 2,
        'currency': DEMO_CURRENCY,
        'index_stub': DEMO_RATE_INDEX_STUB
    }
    vanilla_static = QuantLibBondStaticBase.from_dict(vanilla_params_dict)

    callable_params_dict = {
        'valuation_date': val_d, 'maturity_date': call_mat,
        'coupon_rate': coupon_rate_generic, 'face_value': face_value_generic,
        'call_dates': call_dates_example, 'call_prices': call_prices_example,
        'settlement_days': 0, 'freq': 2,
        'currency': DEMO_CURRENCY,
        'index_stub': DEMO_RATE_INDEX_STUB
    }
    callable_static = CallableBondStaticBase.from_dict(callable_params_dict)

    convertible_params_dict = {
        'valuation_date': val_d, 'issue_date': issue_date_conv_ex, 'maturity_date': conv_mat,
        'coupon_rate': 0.02, 'conversion_ratio': CONV_CONVERSION_RATIO,
        'dividend_yield': CONV_DIV_YIELD, 'equity_volatility': CONV_VOL_BASE,
        'initial_stock_price': CONV_S0_BASE, 'credit_spread_value': CONV_CREDIT_SPREAD,
        'face_value': face_value_generic, 'settlement_days': 0, 'freq': 2,
        'currency': DEMO_CURRENCY,
        'index_stub': DEMO_RATE_INDEX_STUB # Or could be DEMO_EQUITY_STUB if equity related
    }
    convertible_static = ConvertibleBondStaticBase.from_dict(convertible_params_dict)

    option_params_dict = {
        'valuation_date': val_d, 'expiry_date': option_expiry_ex,
        'strike_price': OPT_STRIKE_PRICE, 'option_type': OPT_TYPE_STR,
        'currency': DEMO_CURRENCY
    }
    european_option_static = EuropeanOptionStatic.from_dict(option_params_dict)
    print(f"   Product static objects defined for {DEMO_CURRENCY}.")

    # --- 2. Initializing Pricer Templates ---
    print("\n2. Initializing Pricer Templates...")
    fast_pricer_template = FastBondPricer(vanilla_static)
    ql_pricer_vanilla_template = QuantLibBondPricer(vanilla_static, method='discount')
    ql_pricer_callable_g2_template = QuantLibBondPricer(callable_static, method='g2', grid_steps=G2_MODEL_GRID_STEPS)
    ql_pricer_convertible_template = QuantLibBondPricer(convertible_static, method='convertible_binomial', convertible_engine_steps=CONV_ENGINE_STEPS)
    bs_option_pricer_template = BlackScholesPricer(european_option_static, risk_free_rate=OPT_BS_RISK_FREE_RATE, dividend_yield=OPT_BS_DIVIDEND_YIELD)
    print(f"   Pricer templates initialized (G2 grid: {G2_MODEL_GRID_STEPS}, Conv grid: {CONV_ENGINE_STEPS}).")

    # --- 3. Market Data & Scenarios (Using new ScenarioGenerator config) ---
    print(f"\n3. Market Scenarios for TFFs (Mainly for {DEMO_CURRENCY} factors)")

    numeric_rate_tenors = np.array([0.25, 0.5, 1.0, 2.0, 3.0, 5.0, 7.0, 10.0], dtype=float)
    base_demo_currency_rates_values = np.array([0.020, 0.021, 0.022, 0.025, 0.027, 0.030, 0.032, 0.033], dtype=float)

    # Define Factor Maps for ScenarioGenerator
    base_rates_map = {}
    demo_currency_rate_factor_names = [] # Will store ["USD_IR_0.25Y", ...]
    for i, tenor in enumerate(numeric_rate_tenors):
        factor_name = f"{DEMO_CURRENCY}_{DEMO_RATE_INDEX_STUB}_{tenor:.2f}Y"
        base_rates_map[factor_name] = base_demo_currency_rates_values[i]
        demo_currency_rate_factor_names.append(factor_name)

    demo_currency_s0_factor_name = f"{DEMO_CURRENCY}_{DEMO_EQUITY_STUB}_S0"
    demo_currency_vol_factor_name = f"{DEMO_CURRENCY}_{DEMO_EQUITY_STUB}_VOL"

    base_s0_map = {demo_currency_s0_factor_name: CONV_S0_BASE}
    s0_shock_config_map = {demo_currency_s0_factor_name: ('uniform', SIMULATION_VARIANCE_FACTOR_S0_PERCENT)}
    base_vol_map = {demo_currency_vol_factor_name: CONV_VOL_BASE}
    vol_shock_config_map = {demo_currency_vol_factor_name: ('uniform', SIMULATION_VARIANCE_FACTOR_VOL_PERCENT)}

    # Instantiate Scenario Generator
    scenario_gen_for_tff = SimpleRandomScenarioGenerator(
        base_rates_map=base_rates_map,
        base_s0_map=base_s0_map,
        s0_shock_config_map=s0_shock_config_map,
        base_vol_map=base_vol_map,
        vol_shock_config_map=vol_shock_config_map,
        default_rate_shock_std_dev=SIMULATION_VARIANCE_FACTOR_RATES,
        random_seed=42
    )
    full_raw_market_scenarios, full_raw_factor_names = scenario_gen_for_tff.generate_scenarios(N_SCENARIOS_FOR_TFF_DOMAIN)
    print(f"   Generated {full_raw_market_scenarios.shape[1]} factors for {N_SCENARIOS_FOR_TFF_DOMAIN} scenarios.")
    print(f"   Generated factor names: {full_raw_factor_names}")

    # Prepare scenario slices based on new factor names
    indices_demo_currency_rates = [full_raw_factor_names.index(name) for name in demo_currency_rate_factor_names if name in full_raw_factor_names]
    scens_demo_currency_rates_only = full_raw_market_scenarios[:, indices_demo_currency_rates]

    conv_tff_input_factor_names_demo_currency = demo_currency_rate_factor_names.copy()
    scens_cv_tff_raw_demo_currency = scens_demo_currency_rates_only.copy()
    if bond_tff_for_convertible_includes_s0 and demo_currency_s0_factor_name in full_raw_factor_names:
        conv_tff_input_factor_names_demo_currency.append(demo_currency_s0_factor_name)
        s0_col_idx = full_raw_factor_names.index(demo_currency_s0_factor_name)
        scens_cv_tff_raw_demo_currency = np.hstack((scens_cv_tff_raw_demo_currency, full_raw_market_scenarios[:, s0_col_idx, np.newaxis]))

    opt_tff_input_factor_names_demo_currency = []
    scens_opt_tff_raw_demo_currency = None
    if option_tff_factors_s0_vol and demo_currency_s0_factor_name in full_raw_factor_names and demo_currency_vol_factor_name in full_raw_factor_names:
        opt_tff_input_factor_names_demo_currency = [demo_currency_s0_factor_name, demo_currency_vol_factor_name]
        s0_col_idx = full_raw_factor_names.index(demo_currency_s0_factor_name)
        vol_col_idx = full_raw_factor_names.index(demo_currency_vol_factor_name)
        scens_opt_tff_raw_demo_currency = full_raw_market_scenarios[:, [s0_col_idx, vol_col_idx]]

    ql_evaluation_date = ql.Date(val_d.day, val_d.month, val_d.year)
    ql.Settings.instance().evaluationDate = ql_evaluation_date

    # --- 4. Single Product Prices (Base Case) ---
    print(f"\n4. Single Product Prices ({DEMO_CURRENCY} Base Case):")
    base_curve_for_pricer = np.array([base_rates_map[name] for name in demo_currency_rate_factor_names])

    print(f"   Vanilla ({DEMO_CURRENCY}, Fast): {fast_pricer_template.price(pillar_times=numeric_rate_tenors, market_scenario_data=base_curve_for_pricer)[0]:.6f}")
    print(f"   Vanilla ({DEMO_CURRENCY}, QL)  : {ql_pricer_vanilla_template.price(pillar_times=numeric_rate_tenors, market_scenario_data=base_curve_for_pricer)[0]:.6f}")

    base_ts_handle = ql_pricer_callable_g2_template._make_term_structure(numeric_rate_tenors, base_curve_for_pricer)
    # Assuming Euribor3M for G2 calibration, adjust if DEMO_RATE_INDEX_STUB implies something else
    # For a generic DEMO_RATE_INDEX_STUB = "IR", we might need a generic IborIndex if G2 calibration is active.
    # If DEMO_CURRENCY is USD and DEMO_RATE_INDEX_STUB is SOFR, it would be ql.Sofr(base_ts_handle)
    # For simplicity, let's keep Euribor3M if G2 calibration is attempted, or ensure it's tied to product's index_stub
    underlying_idx_for_g2 = ql.Euribor3M(base_ts_handle) if DEMO_CURRENCY == "EUR" else ql.USDLibor(ql.Period("3M"), base_ts_handle) # Example
    if callable_static.index_stub == "SOFR" and DEMO_CURRENCY == "USD": # Example of specific index
         underlying_idx_for_g2 = ql.Sofr(base_ts_handle)


    g2_calibrator = G2Calibrator(base_ts_handle, underlying_idx_for_g2)
    cap_tenors = [ql.Period(y, ql.Years) for y in [1, 2, 3, 5, 7, 10]]
    cap_vols = [0.0120, 0.0125, 0.0128, 0.0130, 0.0135, 0.0140] # Example vols
    cap_quote_handles = [ql.QuoteHandle(ql.SimpleQuote(v)) for v in cap_vols]
    calibrated_g2_params = (0.01, 0.003, 0.015, 0.006, -0.75) # Default hardcoded
    if not use_hardcoded_g2_params:
        print("   Attempting G2 calibration for single curve price...")
        try:
            calibrated_g2_params = g2_calibrator.calibrate(cap_tenors, cap_quote_handles, engine_steps=G2_CALIBRATION_ENGINE_STEPS)
        except Exception as e_cal:
            print(f"   G2 Calibration failed: {e_cal}. Using hardcoded fallback.")
    print(f"   G2 Params (single)  : {calibrated_g2_params} ({'Hardcoded' if use_hardcoded_g2_params else 'Calibrated'})")

    base_conv_market_data_for_ql = base_curve_for_pricer # For QL pricer, S0 is handled internally via process
    if bond_tff_for_convertible_includes_s0 : # For TFF scenario data structure consistency
         # The QL pricer itself for convertible bonds takes S0 via the equity process, not in market_scenario_data directly.
         # market_scenario_data for QL convertible pricer should just be rates.
         # S0 for QL convertible is part of the ConvertibleBondStatic.initial_stock_price and then overridden by
         # the S0 value passed to the _price_single_curve_logic -> _price_convertible_static's s0_scen.
         # For base price:
         pass # S0 is taken from convertible_static.initial_stock_price

    print(f"   Callable ({DEMO_CURRENCY}, G2): {ql_pricer_callable_g2_template.price(numeric_rate_tenors, base_curve_for_pricer, g2_params=calibrated_g2_params)[0]:.6f}")
    # QLConvertiblePricer.price expects market_scenario_data to be rates, or rates + S0 if S0 is a TFF factor for it.
    # For the base price call, we can pass just rates; S0 comes from convertible_static.initial_stock_price.
    # If S0 was also a TFF factor, we'd append CONV_S0_BASE here for the base price.
    base_price_conv_inputs = base_curve_for_pricer
    if convertible_static.initial_stock_price is not None and bond_tff_for_convertible_includes_s0:
        # If S0 is a TFF factor, the pricer expects it in the scenario data
        base_price_conv_inputs = np.append(base_curve_for_pricer, convertible_static.initial_stock_price)

    print(f"   Convertible ({DEMO_CURRENCY}, Binom): {ql_pricer_convertible_template.price(numeric_rate_tenors, base_price_conv_inputs)[0]:.6f}")
    print(f"   EurOption ({DEMO_CURRENCY}, BS): {bs_option_pricer_template.price(stock_price=CONV_S0_BASE, volatility=CONV_VOL_BASE):.6f}")

    # --- Parallel Worker Setting ---
    parallel_workers_setting = None if enable_parallel_tff_fitting else False
    if enable_parallel_tff_fitting and parallel_workers_setting is None:
        try:
            num_cores = os.cpu_count() # May return None
            parallel_workers_setting = min(4, num_cores) if num_cores and num_cores > 0 else 1 # Cap at 4 for demo
        except NotImplementedError:
            parallel_workers_setting = 1 # Fallback
        print(f"   Using {parallel_workers_setting} workers for parallel TFF fitting.")


    # --- TFF Fitting & Caching ---
    portfolio_manager = Portfolio()
    tff_fit_times = {}
    tff_eval_times = {}
    tff_total_times = {}
    ql_extrapolated_times = {}

    # Vanilla Bond TFF
    print(f"\n6. TFF Fitting (Vanilla {DEMO_CURRENCY} Bond):")
    vanilla_tff_id = f"{DEMO_CURRENCY}_{DEMO_RATE_INDEX_STUB}_VANILLA_5Y"
    base_values_vanilla_tff = np.array([base_rates_map[name] for name in demo_currency_rate_factor_names])
    tff_cal_vanilla = TensorFunctionalFormCalibrate(
        ql_pricer_vanilla_template, demo_currency_rate_factor_names, base_values_vanilla_tff
    )
    s_t = time.time()
    model_tff_vanilla, _, _, rmse_vanilla, norm_params_v = tff_cal_vanilla.sample_and_fit(
        scens_demo_currency_rates_only, n_train=64, n_test=50, random_seed=43, parallel_workers=parallel_workers_setting
    )
    tff_fit_times['vanilla'] = time.time() - s_t
    portfolio_manager.cache_tff_model(vanilla_tff_id, model_tff_vanilla, demo_currency_rate_factor_names, norm_params_v)
    print(f"   {DEMO_CURRENCY} Vanilla TFF Fit Time: {tff_fit_times['vanilla']:.2f}s, RMSE: {rmse_vanilla:.6f}")

    # Callable Bond TFF
    print(f"\n7. TFF Fitting (Callable G2 {DEMO_CURRENCY} Bond):")
    callable_tff_id = f"{DEMO_CURRENCY}_{DEMO_RATE_INDEX_STUB}_CALLABLE_5Y_G2"
    base_values_callable_tff = np.array([base_rates_map[name] for name in demo_currency_rate_factor_names])
    tff_cal_callable = TensorFunctionalFormCalibrate(
        ql_pricer_callable_g2_template, demo_currency_rate_factor_names, base_values_callable_tff
    )
    s_t = time.time()
    model_tff_callable, _, _, rmse_callable, norm_params_c = tff_cal_callable.sample_and_fit(
        scens_demo_currency_rates_only, n_train=64, n_test=16, random_seed=44,
        g2_params=calibrated_g2_params, parallel_workers=parallel_workers_setting
    )
    tff_fit_times['callable'] = time.time() - s_t
    portfolio_manager.cache_tff_model(callable_tff_id, model_tff_callable, demo_currency_rate_factor_names, norm_params_c)
    print(f"   {DEMO_CURRENCY} Callable TFF Fit Time: {tff_fit_times['callable']:.2f}s, RMSE: {rmse_callable:.6f}")

    # Convertible Bond TFF
    print(f"\n7a. TFF Fitting (Convertible {DEMO_CURRENCY} Bond):")
    conv_tff_id_suffix = "_S0FACTOR" if bond_tff_for_convertible_includes_s0 else ""
    convertible_tff_id = f"{DEMO_CURRENCY}_{DEMO_EQUITY_STUB}_CONV_BOND_5Y{conv_tff_id_suffix}"
    # conv_tff_input_factor_names_demo_currency and scens_cv_tff_raw_demo_currency are already prepared
    base_values_conv_tff = np.array(
        [base_rates_map[name] for name in demo_currency_rate_factor_names] +
        ([base_s0_map[demo_currency_s0_factor_name]] if bond_tff_for_convertible_includes_s0 and demo_currency_s0_factor_name in base_s0_map else [])
    )
    tff_cal_convertible = TensorFunctionalFormCalibrate(
        ql_pricer_convertible_template, conv_tff_input_factor_names_demo_currency, base_values_conv_tff,
        base_s0_for_convertible_tff=CONV_S0_BASE if bond_tff_for_convertible_includes_s0 else None
    )
    s_t = time.time()
    model_tff_convertible, _, _, rmse_convertible, norm_params_cv = tff_cal_convertible.sample_and_fit(
        scens_cv_tff_raw_demo_currency, n_train=64, n_test=16, random_seed=45, parallel_workers=parallel_workers_setting
    )
    tff_fit_times['convertible'] = time.time() - s_t
    portfolio_manager.cache_tff_model(convertible_tff_id, model_tff_convertible, conv_tff_input_factor_names_demo_currency, norm_params_cv)
    print(f"   {DEMO_CURRENCY} Convertible TFF Fit Time: {tff_fit_times['convertible']:.2f}s, RMSE: {rmse_convertible:.6f} (S0 factor: {bond_tff_for_convertible_includes_s0})")

    # European Option TFF
    model_tff_option, norm_params_option = None, None # Initialize
    if option_tff_factors_s0_vol and scens_opt_tff_raw_demo_currency is not None:
        print(f"\n7b. TFF Fitting (European Option {DEMO_CURRENCY} Underlying - Eng.Feat.Order {option_feature_eng_order}):")
        option_tff_id = f'{DEMO_CURRENCY}_{DEMO_EQUITY_STUB}_EURO_CALL_1Y_STRIKE{OPT_STRIKE_PRICE}_ORD{option_feature_eng_order}'
        # opt_tff_input_factor_names_demo_currency and scens_opt_tff_raw_demo_currency are prepared
        base_values_option_tff = np.array([
            base_s0_map[demo_currency_s0_factor_name], base_vol_map[demo_currency_vol_factor_name]
        ])
        tff_cal_option = TensorFunctionalFormCalibrate(
            bs_option_pricer_template, opt_tff_input_factor_names_demo_currency, base_values_option_tff
        )
        s_t = time.time()
        model_tff_option, _, _, rmse_option, norm_params_option = tff_cal_option.sample_and_fit(
            scens_opt_tff_raw_demo_currency, n_train=128, n_test=50, random_seed=46,
            parallel_workers=parallel_workers_setting, option_feature_order=option_feature_eng_order
        )
        tff_fit_times['option'] = time.time() - s_t
        portfolio_manager.cache_tff_model(
            option_tff_id, model_tff_option,
            opt_tff_input_factor_names_demo_currency, norm_params_option, option_feature_eng_order
        )
        print(f"   {DEMO_CURRENCY} Option TFF Fit Time: {tff_fit_times['option']:.2f}s, RMSE: {rmse_option:.6f}")

    # --- 8. Performance Benchmarking ---
    print(f"\n8. Performance Benchmarking ({DEMO_CURRENCY} factors):")
    n_subset = max(1, int(N_SCENARIOS_FOR_TFF_DOMAIN * N_SCENARIOS_FOR_BENCHMARK_SUBSET_PERCENT))
    extrapolation_factor = N_SCENARIOS_FOR_TFF_DOMAIN / n_subset if n_subset > 0 else N_SCENARIOS_FOR_TFF_DOMAIN
    print(f"   Pricing {n_subset} scenarios for QL/BS timing, then extrapolating by {extrapolation_factor:.1f}x for {N_SCENARIOS_FOR_TFF_DOMAIN} scenarios.")

    # TFF Evaluation Times (on full set)
    s_t=time.time(); model_tff_vanilla(scens_demo_currency_rates_only); tff_eval_times['vanilla'] = time.time()-s_t
    s_t=time.time(); model_tff_callable(scens_demo_currency_rates_only); tff_eval_times['callable'] = time.time()-s_t
    s_t=time.time(); model_tff_convertible(scens_cv_tff_raw_demo_currency); tff_eval_times['convertible'] = time.time()-s_t
    if model_tff_option and scens_opt_tff_raw_demo_currency is not None:
        opt_inputs_for_eval = scens_opt_tff_raw_demo_currency
        if norm_params_option and norm_params_option.get('is_engineered', False):
            eng_feat, _ = engineer_option_features(scens_opt_tff_raw_demo_currency[:,0], scens_opt_tff_raw_demo_currency[:,1], order=option_feature_eng_order)
            opt_inputs_for_eval, _, _ = normalize_features(eng_feat, norm_params_option['means'], norm_params_option['stds'])
        s_t=time.time(); model_tff_option(opt_inputs_for_eval); tff_eval_times['option'] = time.time()-s_t

    # QL/BS Timings (on subset, then extrapolate)
    subset_rates_scens = scens_demo_currency_rates_only[:n_subset]
    subset_conv_scens = scens_cv_tff_raw_demo_currency[:n_subset] if scens_cv_tff_raw_demo_currency is not None else None
    subset_opt_scens = scens_opt_tff_raw_demo_currency[:n_subset] if scens_opt_tff_raw_demo_currency is not None else None

    s_t=time.time(); ql_pricer_vanilla_template.price(numeric_rate_tenors, subset_rates_scens); time_ql_v_sub = time.time()-s_t
    ql_extrapolated_times['vanilla'] = time_ql_v_sub * extrapolation_factor

    g2_params_bench = calibrated_g2_params # Use already calibrated/hardcoded
    s_t=time.time(); ql_pricer_callable_g2_template.price(numeric_rate_tenors, subset_rates_scens, g2_params=g2_params_bench); time_ql_c_sub = time.time()-s_t
    ql_extrapolated_times['callable'] = time_ql_c_sub * extrapolation_factor

    if subset_conv_scens is not None:
        s_t=time.time(); ql_pricer_convertible_template.price(numeric_rate_tenors, subset_conv_scens); time_ql_cv_sub = time.time()-s_t
        ql_extrapolated_times['convertible'] = time_ql_cv_sub * extrapolation_factor

    if model_tff_option and subset_opt_scens is not None:
        s_t=time.time(); bs_option_pricer_template.price(subset_opt_scens[:,0], subset_opt_scens[:,1]); time_bs_opt_sub = time.time()-s_t
        ql_extrapolated_times['option'] = time_bs_opt_sub * extrapolation_factor

    print(f"   TFF Eval Vanilla: {tff_eval_times.get('vanilla',0):.4f}s | Extrap. QL Vanilla: {ql_extrapolated_times.get('vanilla',0):.2f}s")
    print(f"   TFF Eval Callable: {tff_eval_times.get('callable',0):.4f}s | Extrap. QL G2: {ql_extrapolated_times.get('callable',0):.2f}s")
    print(f"   TFF Eval Convertible: {tff_eval_times.get('convertible',0):.4f}s | Extrap. QL Conv: {ql_extrapolated_times.get('convertible',0):.2f}s")
    if 'option' in tff_eval_times:
        print(f"   TFF Eval Option: {tff_eval_times.get('option',0):.4f}s | Extrap. BS Option: {ql_extrapolated_times.get('option',0):.2f}s")

    for ptype in tff_fit_times:
        tff_total_times[ptype] = tff_fit_times.get(ptype,0) + tff_eval_times.get(ptype,0)
        if ql_extrapolated_times.get(ptype,0) > 0 and tff_total_times[ptype] > 0:
             speedup = ql_extrapolated_times[ptype] / tff_total_times[ptype]
             print(f"   Speedup TFF {ptype.capitalize()} (Total) vs Full Pricer: {speedup:.1f}x")


    # --- 9. Portfolio Demonstration ---
    print(f"\n9. Portfolio Pricing Demonstration ({DEMO_CURRENCY} instruments):")

    vanilla_direct_tff_config = {
        'model_dict': model_tff_vanilla.to_dict(),
        'raw_input_names': demo_currency_rate_factor_names, # Names used for vanilla TFF
        'normalization_params': norm_params_v, # Normalization params for vanilla TFF
        'option_feature_order': 0
    }

    callable_direct_tff_config = {
        'model_dict': model_tff_callable.to_dict(),
        'raw_input_names': demo_currency_rate_factor_names, # Names used for callable TFF
        'normalization_params': norm_params_c, # Normalization params for callable TFF
        'option_feature_order': 0
    }


    # Define portfolio composition with currency-specific instrument IDs
    portfolio_composition_specs = [
        {
            'instrument_id': vanilla_tff_id, # e.g., "USD_IR_VANILLA_5Y"
            'product_static_object': vanilla_static, # This is USD vanilla_static
            'num_holdings': 500,
            'pricing_engine_type': 'tff',
        },
        {
            'instrument_id': callable_tff_id, # e.g., "USD_IR_CALLABLE_5Y_G2"
            'product_static_object': callable_static, # This is USD callable_static
            'num_holdings': 500,
            'pricing_engine_type': 'tff',
            'direct_tff_config': callable_direct_tff_config
        },
        {
            'instrument_id': convertible_tff_id, # e.g., "USD_STOCK_CONV_BOND_5Y_S0FACTOR"
            'product_static_object': convertible_static, # USD convertible_static
            'num_holdings': 500,
            'pricing_engine_type': 'tff',
        },
        {
            'instrument_id': 'USD_VANILLA_BOND_DIRECT_TFF_EXAMPLE', # A new, unique ID for this position
            'product_static_object': vanilla_static, # The same static definition can be used
            'num_holdings': 500,
            'pricing_engine_type': 'tff',
            'direct_tff_config': vanilla_direct_tff_config
        },
    ]

    # Add european options
    portfolio_composition_specs.append({
        'instrument_id': option_tff_id, # e.g. "USD_STOCK_EURO_CALL..."
        'product_static_object': european_option_static, # USD european_option_static
        'num_holdings': 200,
        'pricing_engine_type': 'tff',
    })

    # portfolio_manager already has TFFs cached.
    # Rebuild portfolio for this run to ensure it uses the latest specs
    current_portfolio = Portfolio() # Fresh portfolio for this run

    # Cache models into this fresh portfolio instance again
    if vanilla_tff_id in portfolio_manager.tff_model_cache: # Check if models exist from previous logic
        current_portfolio.cache_tff_model(vanilla_tff_id, model_tff_vanilla, demo_currency_rate_factor_names, norm_params_v)
    if callable_tff_id in portfolio_manager.tff_model_cache and model_tff_callable: # Check if G2 TFF was made
         # Note: the callable example in portfolio_composition_specs uses 'full', so no TFF needed for it there
         # but if you had a TFF version:
         # current_portfolio.cache_tff_model(callable_tff_id, model_tff_callable, demo_currency_rate_factor_names, norm_params_c)
         pass
    if convertible_tff_id in portfolio_manager.tff_model_cache:
        current_portfolio.cache_tff_model(convertible_tff_id, model_tff_convertible, conv_tff_input_factor_names_demo_currency, norm_params_cv)
    if model_tff_option and option_tff_id in portfolio_manager.tff_model_cache:
         current_portfolio.cache_tff_model(option_tff_id, model_tff_option, opt_tff_input_factor_names_demo_currency, norm_params_option, option_feature_eng_order)

    for item_spec in portfolio_composition_specs:
        current_portfolio.add_position(
            instrument_id=item_spec['instrument_id'],
            product_static=item_spec['product_static_object'],
            num_holdings=item_spec['num_holdings'],
            pricing_engine_type=item_spec['pricing_engine_type'],
            full_pricer_instance=item_spec.get('full_pricer_instance'),
            full_pricer_kwargs=item_spec.get('full_pricer_kwargs'),
            direct_tff_config=item_spec.get('direct_tff_config')
        )
    print(f"   Portfolio created with {len(current_portfolio.positions)} positions.")

    # Scenarios for portfolio evaluation - using the same generator for simplicity
    # In a real system, portfolio scenarios might be different (e.g., historical, more factors)
    portfolio_eval_scenarios, portfolio_eval_factor_names = scenario_gen_for_tff.generate_scenarios(100)

    print(f"   Pricing portfolio over {portfolio_eval_scenarios.shape[0]} scenarios with factors: {portfolio_eval_factor_names}...")
    start_t_portfolio_eval = time.time()
    portfolio_values_scenarios = current_portfolio.price_portfolio(
        raw_market_scenarios=portfolio_eval_scenarios,
        scenario_factor_names=portfolio_eval_factor_names,
        portfolio_rate_pillar_times=numeric_rate_tenors # Numeric tenors for rate curves
    )
    time_portfolio_eval = time.time() - start_t_portfolio_eval

    print(f"   Portfolio Pricing time: {time_portfolio_eval:.4f}s")
    if portfolio_values_scenarios.size > 0:
        print(f"   Mean Portfolio Value: {np.mean(portfolio_values_scenarios):.2f}, StdDev: {np.std(portfolio_values_scenarios):.2f}, 99% VaR: {np.percentile(portfolio_values_scenarios, 99):.2f}")
    else:
        print("   Portfolio pricing did not return values.")

    print("\\n--- End of Demonstration ---")


if __name__ == "__main__":
    try:
        print(f"QuantLib version: {ql.__version__}")
        # Import all classes from the modules directly into the main namespace for simplicity in notebook
        # In a package structure, you'd use: from module_name import ClassName

        # Re-executing notebook cells to define classes is assumed if running interactively
        # If this were a standalone script, imports would be at the top.

        run_demonstration(
            enable_parallel_tff_fitting=False, # Set to False for easier debugging, True for speed
            g2_model_grid_steps_param=16,
            convertible_binomial_steps_param=128,
            option_tff_factors_s0_vol=True,
            bond_tff_for_convertible_includes_s0=True,
            option_feature_eng_order=3,
            use_hardcoded_g2_params=True # Set to False to run G2 calibration
        )
    except NameError as e:
        if 'QuantLibBondStaticBase' in str(e) or 'PricerBase' in str(e) or 'TensorFunctionalFormCalibrate' in str(e) \
            or 'SimpleRandomScenarioGenerator' in str(e) or 'Portfolio' in str(e) or 'G2Calibrator' in str(e):
            print(f"ERROR: A required class is not defined. Ensure all notebook cells defining classes are executed. Details: {e}")
        elif 'QuantLib' in str(e) or 'ql' in str(e):
            print("ERROR: QuantLib not found/imported.\nInstall with: !pip install QuantLib-Python")
        else:
            print(f"A NameError occurred: {e}")
            import traceback
            traceback.print_exc()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()

QuantLib version: 1.38
--- FastRiskDemo (Parallel TFF: False, G2 Grid: 16, Conv Grid: 128, OptTFF(S0,Vol): True, ConvBondTFF(Rates,S0): True, OptFeatOrder: 3, G2Calib: Disabled) ---

1. Defining Product Static Objects (Currency: USD)...
   Product static objects defined for USD.

2. Initializing Pricer Templates...
   Pricer templates initialized (G2 grid: 16, Conv grid: 128).

3. Market Scenarios for TFFs (Mainly for USD factors)
   Generated 10 factors for 2000 scenarios.
   Generated factor names: ['USD_IR_0.25Y', 'USD_IR_0.50Y', 'USD_IR_1.00Y', 'USD_IR_10.00Y', 'USD_IR_2.00Y', 'USD_IR_3.00Y', 'USD_IR_5.00Y', 'USD_IR_7.00Y', 'USD_STOCK_S0', 'USD_STOCK_VOL']

4. Single Product Prices (USD Base Case):
   Vanilla (USD, Fast): 102.501691
   Vanilla (USD, QL)  : 100.908359
   G2 Params (single)  : (0.01, 0.003, 0.015, 0.006, -0.75) (Hardcoded)
   Callable (USD, G2): 100.619401
   Convertible (USD, Binom): 1928.323179
   EurOption (USD, BS): 8.407504

6. TFF Fitting (Vanilla USD Bond):
  